In [54]:
import pandas as pd

df = pd.read_csv('/Users/gf26229/Downloads/orders.csv')
df.head()

,tradingDay,timestamp,contract,orderType,entryType,status,low,high,profitCap,profitMultiplier,placedTimestamp,filledTimestamp,closeTimestamp,cancelReason,accountId,closedAt
0,2024-01-02,1704205860000,ES,Long,"9,12,4",Loss,4785.0186,4788.5000,NaN,2.0,NaN,1704205860000,1704206340000,NaN,NaN,NaN
1,2024-01-02,1704216240000,ES,Short,"9,12,4",Loss,4791.0000,4794.1836,NaN,2.0,NaN,1704216240000,1704217020000,NaN,NaN,NaN
2,2024-01-02,1704217080000,ES,Long,"8,13,21,11",Loss,4792.6523,4795.5000,NaN,2.0,NaN,1704217080000,1704217680000,NaN,NaN,NaN
3,2024-01-02,1704217860000,ES,Short,"7,14,21,11",Profit,4789.0000,4792.3090,NaN,2.0,NaN,1704217860000,1704221820000,NaN,NaN,NaN
4,2024-01-02,1704221760000,ES,Short,"9,13",Profit,4782.2500,4785.4146,NaN,2.0,NaN,1704221760000,1704224400000,NaN,NaN,NaN


In [2]:
df = df.sort_values('timestamp', ascending=True)
df.head()

,tradingDay,timestamp,contract,orderType,entryType,status,low,high,profitCap,profitMultiplier,placedTimestamp,filledTimestamp,closeTimestamp,cancelReason,accountId,closedAt
0,2024-01-02,1704205860000,ES,Long,"9,12,4",Loss,4785.0186,4788.5000,NaN,2.0,NaN,1704205860000,1704206340000,NaN,NaN,NaN
1,2024-01-02,1704216240000,ES,Short,"9,12,4",Loss,4791.0000,4794.1836,NaN,2.0,NaN,1704216240000,1704217020000,NaN,NaN,NaN
2,2024-01-02,1704217080000,ES,Long,"8,13,21,11",Loss,4792.6523,4795.5000,NaN,2.0,NaN,1704217080000,1704217680000,NaN,NaN,NaN
3,2024-01-02,1704217860000,ES,Short,"7,14,21,11",Profit,4789.0000,4792.3090,NaN,2.0,NaN,1704217860000,1704221820000,NaN,NaN,NaN
4,2024-01-02,1704221760000,ES,Short,"9,13",Profit,4782.2500,4785.4146,NaN,2.0,NaN,1704221760000,1704224400000,NaN,NaN,NaN


In [3]:
from collections import Counter

# Identify consecutive loss streaks
loss_streak = 0
loss_streaks = []

for status in df['status']:
    if status == 'Loss':
        loss_streak += 1
    else:
        if loss_streak > 0:
            loss_streaks.append(loss_streak)
        loss_streak = 0

# Don't forget the last streak if it ends with losses
if loss_streak > 0:
    loss_streaks.append(loss_streak)

# Count occurrences of each streak length
streak_counts = Counter(loss_streaks)

# Create a summary dataframe
streak_summary = pd.DataFrame(list(streak_counts.items()), columns=['Consecutive Losses', 'Count'])
streak_summary = streak_summary.sort_values('Consecutive Losses')
print(streak_summary)
print(f"\nTotal number of loss streaks: {len(loss_streaks)}")
print(f"Maximum consecutive losses: {max(loss_streaks) if loss_streaks else 0}")

    Consecutive Losses  Count
1                    1    187
2                    2    139
0                    3     74
3                    4     41
5                    5     30
4                    6     14
9                    7     11
7                    8     12
6                    9      5
10                  10      4
12                  12      1
11                  13      1
8                   17      1

Total number of loss streaks: 520
Maximum consecutive losses: 17


In [8]:
# Calculate capital required for martingale strategy
# Profit target: $1000, Profit to loss ratio: 2:1

capital_required = []

for streak_length in streak_summary['Consecutive Losses']:
    total_losses = 0
    
    # Calculate cumulative losses
    for i in range(streak_length):
        if i == 0:
            risk = 500  # First position risks $500
        else:
            # Double the previous position size
            risk = 500 * (2 ** i)
        total_losses += risk
    
    # Final winning position needs same size as last loss to maintain doubling pattern
    final_risk = 500 * (2 ** streak_length)
    
    # Total capital = all losses + final winning position risk
    total_capital = total_losses + final_risk
    capital_required.append(total_capital)

streak_summary['Capital Required (Martingale)'] = capital_required
print(streak_summary[['Consecutive Losses', 'Count', 'Capital Required (Martingale)']])


    Consecutive Losses  Count  Capital Required (Martingale)
1                    1    187                           1500
2                    2    139                           3500
0                    3     74                           7500
3                    4     41                          15500
5                    5     30                          31500
4                    6     14                          63500
9                    7     11                         127500
7                    8     12                         255500
6                    9      5                         511500
10                  10      4                        1023500
12                  12      1                        4095500
11                  13      1                        8191500
8                   17      1                      131071500


## Analysis: Impact of Loss Streak Management Strategies

Compare different strategies for managing long losing streaks. Baseline: -$1000 per loss, +$2000 per win.

In [11]:
# Strategy 1: Baseline (no changes)
# -$1000 per loss, +$2000 per win

baseline_pnl = []
cumulative = 0

for status in df['status']:
    if status == 'Loss':
        pnl = -1000
    else:  # Win
        pnl = 2000
    cumulative += pnl
    baseline_pnl.append(cumulative)

df['baseline_cumulative'] = baseline_pnl

# Calculate proper drawdown (from peak, not from zero)
df['baseline_peak'] = df['baseline_cumulative'].cummax()
df['baseline_drawdown'] = df['baseline_cumulative'] - df['baseline_peak']
max_drawdown = df['baseline_drawdown'].min()

print(f"Baseline Strategy:")
print(f"Final P&L: ${df['baseline_cumulative'].iloc[-1]:,.2f}")
print(f"Peak: ${df['baseline_peak'].max():,.2f}")
print(f"Maximum Drawdown (from peak): ${max_drawdown:,.2f}")
print(f"Maximum Drawdown %: {(max_drawdown / df['baseline_peak'].max() * 100):.2f}%")

Baseline Strategy:
Final P&L: $327,000.00
Peak: $331,000.00
Maximum Drawdown (from peak): $-38,000.00
Maximum Drawdown %: -11.48%


In [15]:
# Analyze the 17-loss streak and other long streaks
# Let's find where these long streaks occurred and their impact

streak_analysis = []
current_streak = 0
streak_start_idx = None
streak_start_cumulative = None

for idx, status in enumerate(df['status']):
    if status == 'Loss':
        if current_streak == 0:
            streak_start_idx = idx
            streak_start_cumulative = df['baseline_cumulative'].iloc[idx-1] if idx > 0 else 0
        current_streak += 1
    else:
        if current_streak > 0:
            streak_end_cumulative = df['baseline_cumulative'].iloc[idx-1]
            drawdown = streak_end_cumulative - streak_start_cumulative
            streak_analysis.append({
                'Streak Length': current_streak,
                'Start Index': streak_start_idx,
                'End Index': idx - 1,
                'Starting Cumulative': streak_start_cumulative,
                'Ending Cumulative': streak_end_cumulative,
                'Drawdown': drawdown
            })
        current_streak = 0

# Don't forget last streak
if current_streak > 0:
    streak_end_cumulative = df['baseline_cumulative'].iloc[-1]
    drawdown = streak_end_cumulative - streak_start_cumulative
    streak_analysis.append({
        'Streak Length': current_streak,
        'Start Index': streak_start_idx,
        'End Index': len(df) - 1,
        'Starting Cumulative': streak_start_cumulative,
        'Ending Cumulative': streak_end_cumulative,
        'Drawdown': drawdown
    })

streak_df = pd.DataFrame(streak_analysis)
streak_df = streak_df.sort_values('Streak Length', ascending=False)

print("Top 10 Longest Losing Streaks and Their Impact:")
print("="*90)
print(streak_df.head(10).to_string(index=False))
print("="*90)
print(f"\nThe 17-loss streak caused a ${abs(streak_df.iloc[0]['Drawdown']):,.0f} drawdown")
print(f"It started when cumulative P&L was ${streak_df.iloc[0]['Starting Cumulative']:,.0f}")
print(f"It ended when cumulative P&L was ${streak_df.iloc[0]['Ending Cumulative']:,.0f}")

Top 10 Longest Losing Streaks and Their Impact:
 Streak Length  Start Index  End Index  Starting Cumulative  Ending Cumulative  Drawdown
            17          168        184                18000               1000    -17000
            13          887        899                67000              54000    -13000
            12         1709       1720               238000             226000    -12000
            10         1748       1757               238000             228000    -10000
            10         2103       2112               330000             320000    -10000
            10          538        547                38000              28000    -10000
            10         1231       1240               158000             148000    -10000
             9         2160       2168               327000             318000     -9000
             9         2130       2138               327000             318000     -9000
             9          398        406                28000   

In [16]:
# Calculate proper drawdown (from peak)
df['peak'] = df['baseline_cumulative'].cummax()
df['drawdown'] = df['baseline_cumulative'] - df['peak']

max_drawdown = df['drawdown'].min()

print(f"\nProper Drawdown Analysis:")
print(f"Maximum Drawdown (from peak): ${max_drawdown:,.2f}")
print(f"Maximum Drawdown %: {(max_drawdown / df['peak'].max() * 100):.2f}%")

# Find when the max drawdown occurred
max_dd_idx = df['drawdown'].idxmin()
print(f"\nMax drawdown occurred at trade #{max_dd_idx + 1}")
print(f"Peak before drawdown: ${df['peak'].iloc[max_dd_idx]:,.2f}")
print(f"Cumulative P&L at max drawdown: ${df['baseline_cumulative'].iloc[max_dd_idx]:,.2f}")
print(f"Actual drawdown amount: ${max_drawdown:,.2f}")


Proper Drawdown Analysis:
Maximum Drawdown (from peak): $-38,000.00
Maximum Drawdown %: -11.48%

Max drawdown occurred at trade #250
Peak before drawdown: $25,000.00
Cumulative P&L at max drawdown: $-13,000.00
Actual drawdown amount: $-38,000.00


In [18]:
# Strategy 2: Position Size Reduction After Losses
# Reduce position size by 50% after 3 consecutive losses
# Continue reducing by 50% for each additional loss
# Reset to full size on a win

strategy2_pnl = []
cumulative = 0
current_streak = 0
base_loss = -1000
base_win = 2000

for status in df['status']:
    if status == 'Loss':
        current_streak += 1
        # Start reducing after 3 losses
        if current_streak >= 3:
            reduction_factor = 0.5 ** (current_streak - 2)
        else:
            reduction_factor = 1.0
        pnl = base_loss * reduction_factor
    else:  # Win
        # Apply same reduction to wins during a streak
        if current_streak >= 3:
            reduction_factor = 0.5 ** (current_streak - 2)
        else:
            reduction_factor = 1.0
        pnl = base_win * reduction_factor
        current_streak = 0  # Reset on win
    
    cumulative += pnl
    strategy2_pnl.append(cumulative)

df['strategy2_cumulative'] = strategy2_pnl

# Calculate proper drawdown (from peak)
df['strategy2_peak'] = df['strategy2_cumulative'].cummax()
df['strategy2_drawdown'] = df['strategy2_cumulative'] - df['strategy2_peak']
strategy2_max_dd = df['strategy2_drawdown'].min()

print(f"\nStrategy 2: Position Size Reduction")
print(f"Final P&L: ${df['strategy2_cumulative'].iloc[-1]:,.2f}")
print(f"Peak: ${df['strategy2_peak'].max():,.2f}")
print(f"Maximum Drawdown (from peak): ${strategy2_max_dd:,.2f}")
print(f"Maximum Drawdown %: {(strategy2_max_dd / df['strategy2_peak'].max() * 100):.2f}%")
print(f"Difference from Baseline: ${df['strategy2_cumulative'].iloc[-1] - df['baseline_cumulative'].iloc[-1]:,.2f}")


Strategy 2: Position Size Reduction
Final P&L: $435,887.30
Peak: $438,637.30
Maximum Drawdown (from peak): $-15,468.66
Maximum Drawdown %: -3.53%
Difference from Baseline: $108,887.30


In [19]:
# Strategy 3: Maximum Consecutive Loss Limit
# Stop trading after 5 consecutive losses
# Resume on next win

strategy3_pnl = []
cumulative = 0
current_streak = 0
max_consecutive_losses = 5
base_loss = -1000
base_win = 2000

for status in df['status']:
    if status == 'Loss':
        current_streak += 1
        # Only take the loss if under the limit
        if current_streak <= max_consecutive_losses:
            pnl = base_loss
        else:
            pnl = 0  # Skip this trade
    else:  # Win
        if current_streak <= max_consecutive_losses:
            pnl = base_win
        else:
            pnl = 0  # Skip this trade too (we weren't trading)
        current_streak = 0  # Reset on win
    
    cumulative += pnl
    strategy3_pnl.append(cumulative)

df['strategy3_cumulative'] = strategy3_pnl

# Calculate proper drawdown (from peak)
df['strategy3_peak'] = df['strategy3_cumulative'].cummax()
df['strategy3_drawdown'] = df['strategy3_cumulative'] - df['strategy3_peak']
strategy3_max_dd = df['strategy3_drawdown'].min()

print(f"\nStrategy 3: Max Loss Limit (Stop after 5 losses)")
print(f"Final P&L: ${df['strategy3_cumulative'].iloc[-1]:,.2f}")
print(f"Peak: ${df['strategy3_peak'].max():,.2f}")
print(f"Maximum Drawdown (from peak): ${strategy3_max_dd:,.2f}")
print(f"Maximum Drawdown %: {(strategy3_max_dd / df['strategy3_peak'].max() * 100):.2f}%")
print(f"Difference from Baseline: ${df['strategy3_cumulative'].iloc[-1] - df['baseline_cumulative'].iloc[-1]:,.2f}")


Strategy 3: Max Loss Limit (Stop after 5 losses)
Final P&L: $368,000.00
Peak: $372,000.00
Maximum Drawdown (from peak): $-29,000.00
Maximum Drawdown %: -7.80%
Difference from Baseline: $41,000.00


In [20]:
# Strategy 4: Skip N Trades After Each Loss
# Wait for 1 trade opportunity after each loss before re-entering

strategy4_pnl = []
cumulative = 0
skip_count = 0
base_loss = -1000
base_win = 2000

for status in df['status']:
    if skip_count > 0:
        # Skipping this trade
        skip_count -= 1
        pnl = 0
    elif status == 'Loss':
        pnl = base_loss
        skip_count = 1  # Skip next trade
    else:  # Win
        pnl = base_win
    
    cumulative += pnl
    strategy4_pnl.append(cumulative)

df['strategy4_cumulative'] = strategy4_pnl

# Calculate proper drawdown (from peak)
df['strategy4_peak'] = df['strategy4_cumulative'].cummax()
df['strategy4_drawdown'] = df['strategy4_cumulative'] - df['strategy4_peak']
strategy4_max_dd = df['strategy4_drawdown'].min()

print(f"\nStrategy 4: Skip 1 Trade After Each Loss")
print(f"Final P&L: ${df['strategy4_cumulative'].iloc[-1]:,.2f}")
print(f"Peak: ${df['strategy4_peak'].max():,.2f}")
print(f"Maximum Drawdown (from peak): ${strategy4_max_dd:,.2f}")
print(f"Maximum Drawdown %: {(strategy4_max_dd / df['strategy4_peak'].max() * 100):.2f}%")
print(f"Difference from Baseline: ${df['strategy4_cumulative'].iloc[-1] - df['baseline_cumulative'].iloc[-1]:,.2f}")


Strategy 4: Skip 1 Trade After Each Loss
Final P&L: $247,000.00
Peak: $250,000.00
Maximum Drawdown (from peak): $-26,000.00
Maximum Drawdown %: -10.40%
Difference from Baseline: $-80,000.00


In [21]:
# Strategy 5: Daily Loss Limit
# Stop trading for the rest of the day after losing $3000
# Assumes timestamp column exists and trades are chronological

strategy5_pnl = []
cumulative = 0
daily_loss = 0
current_date = None
stop_trading_today = False
base_loss = -1000
base_win = 2000

df['date'] = pd.to_datetime(df['timestamp']).dt.date

for idx, row in df.iterrows():
    trade_date = row['date']
    
    # Reset daily tracking at start of new day
    if trade_date != current_date:
        current_date = trade_date
        daily_loss = 0
        stop_trading_today = False
    
    # Check if we should trade
    if stop_trading_today:
        pnl = 0
    elif row['status'] == 'Loss':
        pnl = base_loss
        daily_loss += abs(pnl)
        # Stop if we hit daily limit
        if daily_loss >= 3000:
            stop_trading_today = True
    else:  # Win
        pnl = base_win
    
    cumulative += pnl
    strategy5_pnl.append(cumulative)

df['strategy5_cumulative'] = strategy5_pnl

# Calculate proper drawdown (from peak)
df['strategy5_peak'] = df['strategy5_cumulative'].cummax()
df['strategy5_drawdown'] = df['strategy5_cumulative'] - df['strategy5_peak']
strategy5_max_dd = df['strategy5_drawdown'].min()

print(f"\nStrategy 5: Daily Loss Limit ($3000)")
print(f"Final P&L: ${df['strategy5_cumulative'].iloc[-1]:,.2f}")
print(f"Peak: ${df['strategy5_peak'].max():,.2f}")
print(f"Maximum Drawdown (from peak): ${strategy5_max_dd:,.2f}")
print(f"Maximum Drawdown %: {(strategy5_max_dd / df['strategy5_peak'].max() * 100):.2f}%")
print(f"Difference from Baseline: ${df['strategy5_cumulative'].iloc[-1] - df['baseline_cumulative'].iloc[-1]:,.2f}")


Strategy 5: Daily Loss Limit ($3000)
Final P&L: $-3,000.00
Peak: $-1,000.00
Maximum Drawdown (from peak): $-2,000.00
Maximum Drawdown %: 200.00%
Difference from Baseline: $-330,000.00


In [23]:
# Strategy 6: Combined - Position Size Reduction + Max Loss Limit
# Reduce position size by 50% after 3 consecutive losses
# Stop trading completely after 7 consecutive losses
# Resume at full size on next win

strategy6_pnl = []
cumulative = 0
current_streak = 0
max_consecutive_losses = 7
base_loss = -1000
base_win = 2000

for status in df['status']:
    if status == 'Loss':
        current_streak += 1
        # Stop trading after max consecutive losses
        if current_streak > max_consecutive_losses:
            pnl = 0  # Skip this trade
        # Start reducing after 3 losses
        elif current_streak >= 3:
            reduction_factor = 0.5 ** (current_streak - 2)
            pnl = base_loss * reduction_factor
        else:
            pnl = base_loss
    else:  # Win
        # Apply same reduction to wins during a streak (if we're still trading)
        if current_streak > max_consecutive_losses:
            pnl = 0  # Skip this trade too (we weren't trading)
        elif current_streak >= 3:
            reduction_factor = 0.5 ** (current_streak - 2)
            pnl = base_win * reduction_factor
        else:
            pnl = base_win
        current_streak = 0  # Reset on win
    
    cumulative += pnl
    strategy6_pnl.append(cumulative)

df['strategy6_cumulative'] = strategy6_pnl

# Calculate proper drawdown (from peak)
df['strategy6_peak'] = df['strategy6_cumulative'].cummax()
df['strategy6_drawdown'] = df['strategy6_cumulative'] - df['strategy6_peak']
strategy6_max_dd = df['strategy6_drawdown'].min()

print(f"\nStrategy 6: Combined (Reduction + Stop at 7 losses)")
print(f"Final P&L: ${df['strategy6_cumulative'].iloc[-1]:,.2f}")
print(f"Peak: ${df['strategy6_peak'].max():,.2f}")
print(f"Maximum Drawdown (from peak): ${strategy6_max_dd:,.2f}")
print(f"Maximum Drawdown %: {(strategy6_max_dd / df['strategy6_peak'].max() * 100):.2f}%")
print(f"Difference from Baseline: ${df['strategy6_cumulative'].iloc[-1] - df['baseline_cumulative'].iloc[-1]:,.2f}")
print(f"Difference from Strategy 2: ${df['strategy6_cumulative'].iloc[-1] - df['strategy2_cumulative'].iloc[-1]:,.2f}")


Strategy 6: Combined (Reduction + Stop at 7 losses)
Final P&L: $435,906.25
Peak: $438,656.25
Maximum Drawdown (from peak): $-15,437.50
Maximum Drawdown %: -3.52%
Difference from Baseline: $108,906.25
Difference from Strategy 2: $18.95


## Realistic P&L Analysis with Actual Prices and Fees

Calculate actual profit/loss based on entry/exit prices, whole contracts, and fees.

In [44]:
# Calculate P&L and determine contracts based on risk amount
# ES: $50 per point, $2.88 round-trip fee per contract
# MES: $5 per point, $0.95 round-trip fee per contract
# Logic: Use MES unless we need >=30 contracts, then switch to ES

MES_MULTIPLIER = 5.0
ES_MULTIPLIER = 50.0
MES_FEE = 0.95  # Total round-trip fee
ES_FEE = 2.88   # Total round-trip fee
MES_CUTOFF = 30

def determine_contracts(row, risk_amount):
    """
    Determine contract type and count based on risk amount
    Following the logic from Order.scala determineContracts
    """
    # Risk per MES contract
    price_range = row['high'] - row['low']
    at_risk_per_mes = price_range * MES_MULTIPLIER
    
    # How many MES contracts for this risk amount?
    mes_contracts = int(risk_amount / at_risk_per_mes)
    
    # If we need >= 30 MES, switch to ES
    if mes_contracts >= MES_CUTOFF:
        contracts = round(mes_contracts / 10.0)
        contract_type = 'ES'
        multiplier = ES_MULTIPLIER
        fee = ES_FEE
    else:
        contracts = mes_contracts
        contract_type = 'MES'
        multiplier = MES_MULTIPLIER
        fee = MES_FEE
    
    # Ensure at least 1 contract
    contracts = max(1, contracts)
    
    return contract_type, contracts, multiplier, fee

def calculate_trade_pnl(row, risk_amount):
    """Calculate P&L for a trade given a risk amount"""
    contract_type, contracts, multiplier, fee = determine_contracts(row, risk_amount)
    
    # Calculate the price range
    price_range = row['high'] - row['low']
    
    # Entry, stop loss, and take profit
    if row['orderType'] == 'Long':
        entry = row['high']
        stop_loss = row['low']
        take_profit = entry + 2 * price_range
        
        if row['status'] == 'Profit':
            profit_points = take_profit - entry
        else:  # Loss
            profit_points = stop_loss - entry  # negative
    else:  # Short
        entry = row['low']
        stop_loss = row['high']
        take_profit = entry - 2 * price_range
        
        if row['status'] == 'Profit':
            profit_points = entry - take_profit
        else:  # Loss
            profit_points = entry - stop_loss  # negative
    
    # P&L in dollars
    pnl = profit_points * multiplier * contracts
    
    # Apply fees (fee is already round-trip total)
    pnl_with_fees = pnl - (fee * contracts)
    
    return pnl_with_fees, contract_type, contracts

# Calculate baseline P&L for $1000 risk per trade
baseline_results = df.apply(lambda row: calculate_trade_pnl(row, 1000), axis=1, result_type='expand')
df['baseline_pnl'] = baseline_results[0]
df['baseline_contract_type'] = baseline_results[1]
df['baseline_contracts'] = baseline_results[2]

print("Sample calculations (first 10 trades with $1000 risk):")
print("="*90)
print(df[['orderType', 'status', 'low', 'high', 'baseline_contract_type', 'baseline_contracts', 'baseline_pnl']].head(10).to_string(index=False))
print("="*90)
print(f"\nAverage Win: ${df[df['status']=='Profit']['baseline_pnl'].mean():.2f}")
print(f"Average Loss: ${df[df['status']=='Loss']['baseline_pnl'].mean():.2f}")
print(f"Win/Loss Ratio: {abs(df[df['status']=='Profit']['baseline_pnl'].mean() / df[df['status']=='Loss']['baseline_pnl'].mean()):.2f}")
print(f"MES Contract - Win/Loss Ratio: {abs(df[df['status']=='Profit']['pnl_mes'].mean() / df[df['status']=='Loss']['pnl_mes'].mean()):.2f}")

Sample calculations (first 10 trades with $1000 risk):
orderType status       low      high baseline_contract_type  baseline_contracts  baseline_pnl
     Long   Loss 4785.0186 4788.5000                     ES                   6     -1061.700
    Short   Loss 4791.0000 4794.1836                     ES                   6      -972.360
     Long   Loss 4792.6523 4795.5000                     ES                   7     -1016.855
    Short Profit 4789.0000 4792.3090                     ES                   6      1968.120
    Short Profit 4782.2500 4785.4146                     ES                   6      1881.480
     Long Profit 4767.6680 4771.0000                     ES                   6      1981.920
     Long Profit 4775.2350 4779.0000                     ES                   5      1868.100
    Short Profit 4764.5000 4767.7760                     ES                   6      1948.320
     Long Profit 4750.8003 4754.0000                     ES                   6      1902.540
    S

In [45]:
# Strategy 2 Realistic: Position Size Reduction with dynamic risk amounts
# Start with $1000 risk, reduce by 50% after 3 consecutive losses

def calculate_realistic_strategy2(df, base_risk=1000):
    """
    Calculate Strategy 2 with realistic contract sizing
    base_risk: base dollar amount to risk per trade
    """
    strategy_pnl = []
    cumulative = 0
    current_streak = 0
    
    for idx, row in df.iterrows():
        # Calculate reduction factor based on current streak
        if current_streak >= 3:
            reduction_factor = 0.5 ** (current_streak - 2)
        else:
            reduction_factor = 1.0
        
        # Calculate risk amount for this trade
        risk_amount = base_risk * reduction_factor
        
        # Calculate P&L with proper contract sizing
        trade_pnl, contract_type, contracts = calculate_trade_pnl(row, risk_amount)
        cumulative += trade_pnl
        strategy_pnl.append(cumulative)
        
        # Update streak counter AFTER calculating this trade
        if row['status'] == 'Loss':
            current_streak += 1
        else:  # Win
            current_streak = 0  # Reset on win
    
    return strategy_pnl

# Calculate Strategy 2 with $1000 base risk
df['realistic_strategy2'] = calculate_realistic_strategy2(df, base_risk=1000)

# Calculate drawdowns for Strategy 2
df['realistic_strategy2_peak'] = df['realistic_strategy2'].cummax()
df['realistic_strategy2_dd'] = df['realistic_strategy2'] - df['realistic_strategy2_peak']

print("Realistic Strategy 2 Results (Dynamic Risk - Base $1000):")
print("="*80)
print(f"Final P&L: ${df['realistic_strategy2'].iloc[-1]:,.2f}")
print(f"Max Drawdown: ${df['realistic_strategy2_dd'].min():,.2f}")
print(f"Peak: ${df['realistic_strategy2_peak'].max():,.2f}")
print("="*80)

Realistic Strategy 2 Results (Dynamic Risk - Base $1000):
Final P&L: $258,905.19
Max Drawdown: $-29,393.78
Peak: $262,440.58


In [46]:
# Baseline Realistic: Using baseline_pnl column with cumulative sum
df['baseline_realistic_cumulative'] = df['baseline_pnl'].cumsum()
df['baseline_realistic_peak'] = df['baseline_realistic_cumulative'].cummax()
df['baseline_realistic_dd'] = df['baseline_realistic_cumulative'] - df['baseline_realistic_peak']

# Calculate total fees for baseline
baseline_total_fees = 0
for idx, row in df.iterrows():
    contract_type = row['baseline_contract_type']
    contracts = row['baseline_contracts']
    fee = ES_FEE if contract_type == 'ES' else MES_FEE
    baseline_total_fees += fee * contracts

print("\nBaseline (No Risk Management, Dynamic Risk $1000) Results:")
print("="*80)
print(f"Final P&L: ${df['baseline_realistic_cumulative'].iloc[-1]:,.2f}")
print(f"Max Drawdown: ${df['baseline_realistic_dd'].min():,.2f}")
print(f"Peak: ${df['baseline_realistic_peak'].max():,.2f}")
print(f"Total Fees Paid: ${baseline_total_fees:,.2f}")
print("="*80)

# Calculate total fees for Strategy 2
strategy2_total_fees = 0
current_streak = 0
for idx, row in df.iterrows():
    # Calculate reduction factor based on current streak
    if current_streak >= 3:
        reduction_factor = 0.5 ** (current_streak - 2)
    else:
        reduction_factor = 1.0
    
    # Calculate risk amount for this trade
    risk_amount = 1000 * reduction_factor
    
    # Determine contracts
    contract_type, contracts, multiplier, fee = determine_contracts(row, risk_amount)
    strategy2_total_fees += fee * contracts
    
    # Update streak counter AFTER calculating this trade
    if row['status'] == 'Loss':
        current_streak += 1
    else:  # Win
        current_streak = 0  # Reset on win

# Comparison
print("\nStrategy 2 vs Baseline Comparison:")
print("="*80)
improvement_pnl = df['realistic_strategy2'].iloc[-1] - df['baseline_realistic_cumulative'].iloc[-1]
improvement_dd = df['baseline_realistic_dd'].min() - df['realistic_strategy2_dd'].min()
fee_savings = baseline_total_fees - strategy2_total_fees

print(f"\nBaseline:")
print(f"  Final P&L: ${df['baseline_realistic_cumulative'].iloc[-1]:,.2f}")
print(f"  Max Drawdown: ${df['baseline_realistic_dd'].min():,.2f}")
print(f"  Total Fees: ${baseline_total_fees:,.2f}")

print(f"\nStrategy 2:")
print(f"  Final P&L: ${df['realistic_strategy2'].iloc[-1]:,.2f}")
print(f"  Max Drawdown: ${df['realistic_strategy2_dd'].min():,.2f}")
print(f"  Total Fees: ${strategy2_total_fees:,.2f}")

print(f"\nDifference (Strategy 2 - Baseline):")
print(f"  P&L: ${improvement_pnl:,.2f} ({improvement_pnl / df['baseline_realistic_cumulative'].iloc[-1] * 100:.1f}%)")
print(f"  Drawdown: ${improvement_dd:,.2f} ({improvement_dd / abs(df['baseline_realistic_dd'].min()) * 100:.1f}%)")
print(f"  Fees Saved: ${fee_savings:,.2f} ({fee_savings / baseline_total_fees * 100:.1f}%)")
print("="*80)


Baseline (No Risk Management, Dynamic Risk $1000) Results:
Final P&L: $286,207.38
Max Drawdown: $-40,520.58
Peak: $292,828.57
Total Fees Paid: $36,526.68

Strategy 2 vs Baseline Comparison:

Baseline:
  Final P&L: $286,207.38
  Max Drawdown: $-40,520.58
  Total Fees: $36,526.68

Strategy 2:
  Final P&L: $258,905.19
  Max Drawdown: $-29,393.78
  Total Fees: $32,615.61

Difference (Strategy 2 - Baseline):
  P&L: $-27,302.20 (-9.5%)
  Drawdown: $-11,126.80 (-27.5%)
  Fees Saved: $3,911.07 (10.7%)


### Strategy 2 Aggressive: Higher Risk + More Aggressive Dampening

Test with $2000 base risk and more aggressive position reduction (start reducing after 2 losses instead of 3)

In [48]:
# Strategy 2 Aggressive: $2000 base risk with dampening starting after 2 losses
# More aggressive reduction: 0.5 ** (streak - 1) instead of 0.5 ** (streak - 2)

def calculate_aggressive_strategy2(df, base_risk=2000, start_reduction_at=2):
    """
    Calculate aggressive Strategy 2 with higher risk and earlier dampening
    base_risk: base dollar amount to risk per trade
    start_reduction_at: number of losses before starting to reduce position size
    """
    strategy_pnl = []
    cumulative = 0
    current_streak = 0
    
    for idx, row in df.iterrows():
        # Calculate reduction factor based on current streak
        # Start reducing earlier and more aggressively
        if current_streak >= start_reduction_at:
            reduction_factor = 0.5 ** (current_streak - start_reduction_at + 1)
        else:
            reduction_factor = 1.0
        
        # Calculate risk amount for this trade
        risk_amount = base_risk * reduction_factor
        
        # Calculate P&L with proper contract sizing
        trade_pnl, contract_type, contracts = calculate_trade_pnl(row, risk_amount)
        cumulative += trade_pnl
        strategy_pnl.append(cumulative)
        
        # Update streak counter AFTER calculating this trade
        if row['status'] == 'Loss':
            current_streak += 1
        else:  # Win
            current_streak = 0  # Reset on win
    
    return strategy_pnl

# Calculate Aggressive Strategy 2
df['aggressive_strategy2'] = calculate_aggressive_strategy2(df, base_risk=2000, start_reduction_at=2)

# Calculate drawdowns
df['aggressive_strategy2_peak'] = df['aggressive_strategy2'].cummax()
df['aggressive_strategy2_dd'] = df['aggressive_strategy2'] - df['aggressive_strategy2_peak']

# Calculate baseline with $2000 risk
baseline_2k_results = df.apply(lambda row: calculate_trade_pnl(row, 2000), axis=1, result_type='expand')
df['baseline_2k_pnl'] = baseline_2k_results[0]
df['baseline_2k_cumulative'] = df['baseline_2k_pnl'].cumsum()
df['baseline_2k_peak'] = df['baseline_2k_cumulative'].cummax()
df['baseline_2k_dd'] = df['baseline_2k_cumulative'] - df['baseline_2k_peak']

# Calculate fees for both
baseline_2k_fees = 0
aggressive_fees = 0
current_streak = 0

for idx, row in df.iterrows():
    # Baseline $2000
    contract_type, contracts, multiplier, fee = determine_contracts(row, 2000)
    baseline_2k_fees += fee * contracts
    
    # Aggressive strategy
    if current_streak >= 2:
        reduction_factor = 0.5 ** (current_streak - 2 + 1)
    else:
        reduction_factor = 1.0
    risk_amount = 2000 * reduction_factor
    contract_type, contracts, multiplier, fee = determine_contracts(row, risk_amount)
    aggressive_fees += fee * contracts
    
    # Update streak
    if row['status'] == 'Loss':
        current_streak += 1
    else:
        current_streak = 0

print("Aggressive Strategy 2 Results ($2000 base, reduce after 2 losses):")
print("="*80)
print(f"\nBaseline ($2000 risk):")
print(f"  Final P&L: ${df['baseline_2k_cumulative'].iloc[-1]:,.2f}")
print(f"  Max Drawdown: ${df['baseline_2k_dd'].min():,.2f}")
print(f"  Peak: ${df['baseline_2k_peak'].max():,.2f}")
print(f"  Total Fees: ${baseline_2k_fees:,.2f}")

print(f"\nAggressive Strategy 2:")
print(f"  Final P&L: ${df['aggressive_strategy2'].iloc[-1]:,.2f}")
print(f"  Max Drawdown: ${df['aggressive_strategy2_dd'].min():,.2f}")
print(f"  Peak: ${df['aggressive_strategy2_peak'].max():,.2f}")
print(f"  Total Fees: ${aggressive_fees:,.2f}")

improvement_pnl = df['aggressive_strategy2'].iloc[-1] - df['baseline_2k_cumulative'].iloc[-1]
improvement_dd = df['baseline_2k_dd'].min() - df['aggressive_strategy2_dd'].min()
fee_savings = baseline_2k_fees - aggressive_fees

print(f"\nDifference (Aggressive - Baseline $2000):")
print(f"  P&L: ${improvement_pnl:,.2f} ({improvement_pnl / df['baseline_2k_cumulative'].iloc[-1] * 100:.1f}%)")
print(f"  Drawdown: ${improvement_dd:,.2f} ({improvement_dd / abs(df['baseline_2k_dd'].min()) * 100:.1f}%)")
print(f"  Fees Saved: ${fee_savings:,.2f} ({fee_savings / baseline_2k_fees * 100:.1f}%)")

print(f"\nComparison to Original $1000 Baseline:")
print(f"  P&L vs $1000 Baseline: ${df['aggressive_strategy2'].iloc[-1] - df['baseline_realistic_cumulative'].iloc[-1]:,.2f}")
print(f"  Drawdown vs $1000 Baseline: ${df['aggressive_strategy2_dd'].min() - df['baseline_realistic_dd'].min():,.2f}")
print("="*80)

Aggressive Strategy 2 Results ($2000 base, reduce after 2 losses):

Baseline ($2000 risk):
  Final P&L: $585,608.42
  Max Drawdown: $-82,826.66
  Peak: $599,222.40
  Total Fees: $62,166.11

Aggressive Strategy 2:
  Final P&L: $440,466.10
  Max Drawdown: $-58,025.19
  Peak: $446,161.55
  Total Fees: $47,663.51

Difference (Aggressive - Baseline $2000):
  P&L: $-145,142.32 (-24.8%)
  Drawdown: $-24,801.47 (-29.9%)
  Fees Saved: $14,502.60 (23.3%)

Comparison to Original $1000 Baseline:
  P&L vs $1000 Baseline: $154,258.72
  Drawdown vs $1000 Baseline: $-17,504.61


### Strategy 2 Scaled: Dynamic Risk Based on Account Size

Start with $18,000 capital and scale risk based on account balance:
- Start: $500 risk per trade
- At $50k: increase to $1,000 risk per trade
- At $100k: increase to $2,000 risk per trade

Apply Strategy 2 dampening (reduce after 3 consecutive losses) at each level.

In [49]:
# Strategy 2 Scaled: Start with $18k capital, scale risk based on account balance
# $500 risk until $50k, then $1000 risk until $100k, then $2000 risk
# Apply dampening after 3 consecutive losses at each level

def calculate_scaled_strategy2(df, starting_capital=18000):
    """
    Calculate Strategy 2 with scaled risk based on account balance
    - Start with $500 risk
    - At $50k balance: $1000 risk
    - At $100k balance: $2000 risk
    Apply 50% reduction after 3 consecutive losses
    """
    strategy_pnl = []
    account_balance = starting_capital
    current_streak = 0
    
    for idx, row in df.iterrows():
        # Determine base risk based on account balance
        if account_balance >= 100000:
            base_risk = 2000
        elif account_balance >= 50000:
            base_risk = 1000
        else:
            base_risk = 500
        
        # Apply dampening for losing streaks
        if current_streak >= 3:
            reduction_factor = 0.5 ** (current_streak - 2)
        else:
            reduction_factor = 1.0
        
        risk_amount = base_risk * reduction_factor
        
        # Calculate P&L for this trade
        trade_pnl, contract_type, contracts = calculate_trade_pnl(row, risk_amount)
        account_balance += trade_pnl
        strategy_pnl.append(account_balance - starting_capital)  # Track P&L from start
        
        # Update streak counter
        if row['status'] == 'Loss':
            current_streak += 1
        else:
            current_streak = 0
    
    return strategy_pnl

# Calculate Scaled Strategy 2
df['scaled_strategy2'] = calculate_scaled_strategy2(df, starting_capital=18000)

# Calculate drawdowns
df['scaled_strategy2_peak'] = df['scaled_strategy2'].cummax()
df['scaled_strategy2_dd'] = df['scaled_strategy2'] - df['scaled_strategy2_peak']

# Calculate fees and track risk levels used
scaled_fees = 0
current_streak = 0
account_balance = 18000
risk_level_trades = {'$500': 0, '$1000': 0, '$2000': 0}

for idx, row in df.iterrows():
    # Determine base risk based on account balance
    if account_balance >= 100000:
        base_risk = 2000
        risk_level = '$2000'
    elif account_balance >= 50000:
        base_risk = 1000
        risk_level = '$1000'
    else:
        base_risk = 500
        risk_level = '$500'
    
    risk_level_trades[risk_level] += 1
    
    # Apply dampening
    if current_streak >= 3:
        reduction_factor = 0.5 ** (current_streak - 2)
    else:
        reduction_factor = 1.0
    
    risk_amount = base_risk * reduction_factor
    
    # Calculate trade and fees
    trade_pnl, contract_type, contracts = calculate_trade_pnl(row, risk_amount)
    contract_type, contracts, multiplier, fee = determine_contracts(row, risk_amount)
    scaled_fees += fee * contracts
    
    account_balance += trade_pnl
    
    # Update streak
    if row['status'] == 'Loss':
        current_streak += 1
    else:
        current_streak = 0

print("Scaled Strategy 2 Results (Start $18k → Scale Risk with Balance):")
print("="*80)
print(f"\nFinal P&L: ${df['scaled_strategy2'].iloc[-1]:,.2f}")
print(f"Final Account Balance: ${18000 + df['scaled_strategy2'].iloc[-1]:,.2f}")
print(f"Max Drawdown: ${df['scaled_strategy2_dd'].min():,.2f}")
print(f"Peak P&L: ${df['scaled_strategy2_peak'].max():,.2f}")
print(f"Total Fees: ${scaled_fees:,.2f}")

print(f"\nRisk Levels Used:")
print(f"  $500 risk level: {risk_level_trades['$500']:,} trades")
print(f"  $1000 risk level: {risk_level_trades['$1000']:,} trades")
print(f"  $2000 risk level: {risk_level_trades['$2000']:,} trades")

print(f"\nComparisons:")
print(f"  vs $1000 Baseline P&L: ${df['scaled_strategy2'].iloc[-1] - df['baseline_realistic_cumulative'].iloc[-1]:,.2f}")
print(f"  vs $1000 Baseline DD: ${df['scaled_strategy2_dd'].min() - df['baseline_realistic_dd'].min():,.2f}")
print(f"  vs $1000 Strategy 2 P&L: ${df['scaled_strategy2'].iloc[-1] - df['realistic_strategy2'].iloc[-1]:,.2f}")
print(f"  vs $1000 Strategy 2 DD: ${df['scaled_strategy2_dd'].min() - df['realistic_strategy2_dd'].min():,.2f}")
print(f"  vs $2000 Baseline P&L: ${df['scaled_strategy2'].iloc[-1] - df['baseline_2k_cumulative'].iloc[-1]:,.2f}")
print(f"  vs $2000 Aggressive P&L: ${df['scaled_strategy2'].iloc[-1] - df['aggressive_strategy2'].iloc[-1]:,.2f}")
print("="*80)

Scaled Strategy 2 Results (Start $18k → Scale Risk with Balance):

Final P&L: $309,075.06
Final Account Balance: $327,075.06
Max Drawdown: $-30,071.34
Peak P&L: $316,065.02
Total Fees: $39,503.26

Risk Levels Used:
  $500 risk level: 1,064 trades
  $1000 risk level: 244 trades
  $2000 risk level: 933 trades

Comparisons:
  vs $1000 Baseline P&L: $22,867.68
  vs $1000 Baseline DD: $10,449.24
  vs $1000 Strategy 2 P&L: $50,169.87
  vs $1000 Strategy 2 DD: $-677.56
  vs $2000 Baseline P&L: $-276,533.36
  vs $2000 Aggressive P&L: $-131,391.05


### Strategy: Limited Martingale (4-Attempt Max)

Martingale progression for first 3 losses, then stop for the day:
- Loss 1: $1,000 risk
- Loss 2: $2,000 risk (double)
- Loss 3: $4,000 risk (double again)
- Attempt 4: $8,000 risk (final attempt)
- If loss 4: Stop trading for the day (-$15,000 total)
- Next day: Reset to $1,000 risk

This converts all 1-3 loss streaks into wins, but caps exposure at $15k per bad day.

In [60]:
# Limited Martingale Strategy
# Double risk on each loss (up to 3 losses)
# If 4th attempt fails, stop trading for the rest of the day
# Reset to $1000 risk on next trading day

def calculate_limited_martingale(df):
    """
    Calculate Limited Martingale strategy:
    - Loss 1: $1000 risk
    - Loss 2: $2000 risk
    - Loss 3: $4000 risk
    - Attempt 4: $8000 risk
    - If attempt 4 loses: stop trading for the day
    - Next day: reset to $1000
    """
    strategy_pnl = []
    cumulative = 0
    current_streak = 0
    stop_trading_today = False
    current_date = None
    
    # Add date column if not exists
    if 'date' not in df.columns:
        df['date'] = pd.to_datetime(df['timestamp'], unit='ms').dt.date
    
    for idx, row in df.iterrows():
        trade_date = row['date']
        
        # Reset daily tracking at start of new day
        if trade_date != current_date:
            current_date = trade_date
            current_streak = 0
            stop_trading_today = False
        
        # Skip if we're stopped for the day
        if stop_trading_today:
            # Add zero P&L but keep cumulative
            strategy_pnl.append(cumulative)
            continue
        
        # Determine risk based on current streak
        if current_streak == 0:
            risk_amount = 1000
        elif current_streak == 1:
            risk_amount = 2000
        elif current_streak == 2:
            risk_amount = 4000
        elif current_streak == 3:
            risk_amount = 8000
        else:
            # Should never get here, but safety check
            strategy_pnl.append(cumulative)
            continue
        
        # Calculate P&L for this trade
        trade_pnl, contract_type, contracts = calculate_trade_pnl(row, risk_amount)
        cumulative += trade_pnl
        strategy_pnl.append(cumulative)
        
        # Update streak and check if we need to stop
        if row['status'] == 'Loss':
            current_streak += 1
            # If this was our 4th attempt and it failed, stop for the day
            if current_streak >= 4:
                stop_trading_today = True
        else:  # Win - reset streak
            current_streak = 0
    
    return strategy_pnl

# Calculate Limited Martingale
df['limited_martingale'] = calculate_limited_martingale(df)

# Calculate drawdowns
df['limited_martingale_peak'] = df['limited_martingale'].cummax()
df['limited_martingale_dd'] = df['limited_martingale'] - df['limited_martingale_peak']

# Calculate fees and analyze streak conversions
martingale_fees = 0
current_streak = 0
stop_trading_today = False
current_date = None
streak_conversions = {1: {'converted': 0, 'failed': 0}, 
                     2: {'converted': 0, 'failed': 0}, 
                     3: {'converted': 0, 'failed': 0},
                     4: {'converted': 0, 'failed': 0}}
stopped_days = 0

for idx, row in df.iterrows():
    trade_date = row['date']
    
    # Reset daily tracking at start of new day
    if trade_date != current_date:
        current_date = trade_date
        current_streak = 0
        stop_trading_today = False
    
    # Skip if stopped
    if stop_trading_today:
        continue
    
    # Determine risk
    if current_streak == 0:
        risk_amount = 1000
    elif current_streak == 1:
        risk_amount = 2000
    elif current_streak == 2:
        risk_amount = 4000
    elif current_streak == 3:
        risk_amount = 8000
    else:
        continue
    
    # Calculate fees
    contract_type, contracts, multiplier, fee = determine_contracts(row, risk_amount)
    martingale_fees += fee * contracts
    
    # Track streak outcomes
    if row['status'] == 'Loss':
        current_streak += 1
        if current_streak >= 4:
            stop_trading_today = True
            stopped_days += 1
            streak_conversions[4]['failed'] += 1
    else:  # Win
        if current_streak > 0:
            # This streak was converted to a win
            streak_conversions[current_streak]['converted'] += 1
        current_streak = 0

print("Limited Martingale Results:")
print("="*80)
print(f"\nFinal P&L: ${df['limited_martingale'].iloc[-1]:,.2f}")
print(f"Max Drawdown: ${df['limited_martingale_dd'].min():,.2f}")
print(f"Peak: ${df['limited_martingale_peak'].max():,.2f}")
print(f"Total Fees: ${martingale_fees:,.2f}")

print(f"\nStreak Conversions:")
print(f"  1-loss streaks converted to wins: {streak_conversions[1]['converted']}")
print(f"  2-loss streaks converted to wins: {streak_conversions[2]['converted']}")
print(f"  3-loss streaks converted to wins: {streak_conversions[3]['converted']}")
print(f"  4-loss streaks (stopped trading): {streak_conversions[4]['failed']}")
print(f"  Total days stopped: {stopped_days}")

print(f"\nKey Insights:")
print(f"  Average win per converted 1-loss: ~${(1000*2):,.0f} (recover $1k loss + win $2k)")
print(f"  Average win per converted 2-loss: ~${(1000+2000+4000):,.0f} (recover $3k + win $4k)")  
print(f"  Average win per converted 3-loss: ~${(1000+2000+4000+8000):,.0f} (recover $7k + win $8k)")
print(f"  Cost of each 4-loss failure: -$15,000 (1k+2k+4k+8k in losses)")
print(f"  Win rate needed to break even: ~25% on 4th attempt")
print("="*80)

Limited Martingale Results:

Final P&L: $523,127.92
Max Drawdown: $-131,125.78
Peak: $585,942.66
Total Fees: $67,475.53

Streak Conversions:
  1-loss streaks converted to wins: 202
  2-loss streaks converted to wins: 124
  3-loss streaks converted to wins: 57
  4-loss streaks (stopped trading): 101
  Total days stopped: 101

Key Insights:
  Average win per converted 1-loss: ~$2,000 (recover $1k loss + win $2k)
  Average win per converted 2-loss: ~$7,000 (recover $3k + win $4k)
  Average win per converted 3-loss: ~$15,000 (recover $7k + win $8k)
  Cost of each 4-loss failure: -$15,000 (1k+2k+4k+8k in losses)
  Win rate needed to break even: ~25% on 4th attempt


### Analyzing the $131k Drawdown

Let's identify when the maximum drawdown occurred and explore ways to reduce it.

In [61]:
# Analyze the drawdown - when did it occur and what caused it?
max_dd_idx = df['limited_martingale_dd'].idxmin()
max_dd_value = df['limited_martingale_dd'].min()
peak_before = df['limited_martingale_peak'].iloc[max_dd_idx]
balance_at_dd = df['limited_martingale'].iloc[max_dd_idx]

print("Maximum Drawdown Analysis:")
print("="*80)
print(f"Max Drawdown: ${max_dd_value:,.2f}")
print(f"Occurred at trade #{max_dd_idx + 1} on {pd.to_datetime(df.iloc[max_dd_idx]['timestamp'], unit='ms').strftime('%Y-%m-%d')}")
print(f"Peak before drawdown: ${peak_before:,.2f}")
print(f"Balance at max drawdown: ${balance_at_dd:,.2f}")
print(f"Drawdown %: {(max_dd_value / peak_before * 100):.1f}%")

# Look at trades around the max drawdown (20 trades before and after)
start_idx = max(0, max_dd_idx - 20)
end_idx = min(len(df), max_dd_idx + 20)

print(f"\nTrades around maximum drawdown (#{start_idx+1} to #{end_idx+1}):")
print("="*80)

# Count consecutive stopped days
consecutive_stops = 0
max_consecutive_stops = 0
temp_consecutive = 0
current_date = None
stopped_dates = set()

for idx in range(len(df)):
    row = df.iloc[idx]
    trade_date = row['date']
    
    if trade_date != current_date:
        current_date = trade_date
        day_stopped = False
    
    # Check if this trade would have stopped the day
    # We need to track the streak within the day
    if idx > 0 and df.iloc[idx-1]['date'] == trade_date:
        # Same day, continue tracking
        pass
    
# Let's look at the streak of 4-loss failures
print("\nLooking for clusters of 4-loss failures...")
failure_dates = []
current_streak = 0
stop_trading_today = False
current_date = None

for idx, row in df.iterrows():
    trade_date = row['date']
    
    if trade_date != current_date:
        current_date = trade_date
        current_streak = 0
        stop_trading_today = False
    
    if stop_trading_today:
        continue
    
    if row['status'] == 'Loss':
        current_streak += 1
        if current_streak >= 4:
            stop_trading_today = True
            failure_dates.append((trade_date, idx))

print(f"Total days with 4-loss failures: {len(failure_dates)}")

# Find clusters of failure days
if len(failure_dates) > 0:
    print("\nClusters of failure days (3+ failures within 10 days):")
    for i in range(len(failure_dates) - 2):
        date1, idx1 = failure_dates[i]
        date3, idx3 = failure_dates[i+2]
        days_apart = (date3 - date1).days
        if days_apart <= 10:
            cumulative_at_start = df.iloc[idx1]['limited_martingale']
            cumulative_at_end = df.iloc[idx3]['limited_martingale']
            drawdown = cumulative_at_end - cumulative_at_start
            print(f"  {date1} to {date3} ({days_apart} days): {i+1} to {i+3} failures, P&L change: ${drawdown:,.2f}")

print("="*80)

Maximum Drawdown Analysis:
Max Drawdown: $-131,125.78
Occurred at trade #212 on 2024-03-15
Peak before drawdown: $73,578.88
Balance at max drawdown: $-57,546.89
Drawdown %: -178.2%

Trades around maximum drawdown (#192 to #232):

Looking for clusters of 4-loss failures...
Total days with 4-loss failures: 186

Clusters of failure days (3+ failures within 10 days):
  2024-01-04 to 2024-01-11 (7 days): 1 to 3 failures, P&L change: $16,513.84
  2024-01-09 to 2024-01-12 (3 days): 2 to 4 failures, P&L change: $16,650.24
  2024-01-11 to 2024-01-19 (8 days): 3 to 5 failures, P&L change: $15,664.85
  2024-01-12 to 2024-01-22 (10 days): 4 to 6 failures, P&L change: $8,204.16
  2024-01-19 to 2024-01-29 (10 days): 5 to 7 failures, P&L change: $-24,646.28
  2024-01-29 to 2024-02-05 (7 days): 7 to 9 failures, P&L change: $-21,242.68
  2024-02-02 to 2024-02-09 (7 days): 8 to 10 failures, P&L change: $-30,574.25
  2024-02-05 to 2024-02-15 (10 days): 9 to 11 failures, P&L change: $1,294.15
  2024-02-15

### Drawdown Mitigation Ideas

**Option 1: Scale Down After Multiple Failed Days**
- After 2 failed days within a week, reduce all risk levels by 50%
- After 3 failed days within a week, reduce by 75%
- Reset to full risk after a profitable week

**Option 2: Weekly Loss Limit**
- Stop trading for the week after $30k-$50k in losses
- Resume next week at full or reduced risk

**Option 3: Dynamic Base Risk**
- Start at $500 base risk (instead of $1000)
- Scale up to $1000 after reaching $100k profit
- Scale up to $2000 after reaching $250k profit
- Drop back down after significant drawdown

**Option 4: Hybrid with Strategy 2**
- Use martingale for 1-2 loss streaks
- Switch to dampening (Strategy 2) on 3rd loss instead of doubling to $4k
- Cap at $8k total exposure per losing streak

Let's test Option 1 (scale down after multiple failures):

In [62]:
# Option 1: Scale Down After Multiple Failed Days
# Track failed days in rolling 7-day window
# 2 failures in 7 days: reduce risk by 50%
# 3+ failures in 7 days: reduce risk by 75%
# 7 days without failure: reset to full risk

def calculate_adaptive_martingale(df, base_risk=1000):
    """
    Adaptive Martingale: reduces base risk after multiple failed days
    """
    strategy_pnl = []
    cumulative = 0
    current_streak = 0
    stop_trading_today = False
    current_date = None
    
    # Track failed days with dates
    failed_days = []  # List of dates when we hit 4-loss
    
    # Add date column if not exists
    if 'date' not in df.columns:
        df['date'] = pd.to_datetime(df['timestamp'], unit='ms').dt.date
    
    for idx, row in df.iterrows():
        trade_date = row['date']
        
        # Reset daily tracking at start of new day
        if trade_date != current_date:
            current_date = trade_date
            current_streak = 0
            stop_trading_today = False
            
            # Clean up failed days older than 7 days
            failed_days = [d for d in failed_days if (trade_date - d).days <= 7]
            
            # Determine risk reduction based on recent failures
            recent_failures = len(failed_days)
            if recent_failures >= 3:
                risk_scale = 0.25  # 75% reduction
            elif recent_failures >= 2:
                risk_scale = 0.5   # 50% reduction
            else:
                risk_scale = 1.0   # Full risk
        
        # Skip if we're stopped for the day
        if stop_trading_today:
            strategy_pnl.append(cumulative)
            continue
        
        # Determine risk based on current streak and scale
        if current_streak == 0:
            risk_amount = base_risk * risk_scale
        elif current_streak == 1:
            risk_amount = (base_risk * 2) * risk_scale
        elif current_streak == 2:
            risk_amount = (base_risk * 4) * risk_scale
        elif current_streak == 3:
            risk_amount = (base_risk * 8) * risk_scale
        else:
            strategy_pnl.append(cumulative)
            continue
        
        # Calculate P&L for this trade
        trade_pnl, contract_type, contracts = calculate_trade_pnl(row, risk_amount)
        cumulative += trade_pnl
        strategy_pnl.append(cumulative)
        
        # Update streak and check if we need to stop
        if row['status'] == 'Loss':
            current_streak += 1
            if current_streak >= 4:
                stop_trading_today = True
                if trade_date not in failed_days:
                    failed_days.append(trade_date)
        else:  # Win
            current_streak = 0
    
    return strategy_pnl

# Calculate Adaptive Martingale
df['adaptive_martingale'] = calculate_adaptive_martingale(df, base_risk=1000)

# Calculate drawdowns
df['adaptive_martingale_peak'] = df['adaptive_martingale'].cummax()
df['adaptive_martingale_dd'] = df['adaptive_martingale'] - df['adaptive_martingale_peak']

# Calculate statistics
adaptive_fees = 0
current_streak = 0
stop_trading_today = False
current_date = None
failed_days = []
streak_conversions = {1: {'converted': 0, 'failed': 0}, 
                     2: {'converted': 0, 'failed': 0}, 
                     3: {'converted': 0, 'failed': 0},
                     4: {'converted': 0, 'failed': 0}}
reduced_risk_trades = 0
full_risk_trades = 0

for idx, row in df.iterrows():
    trade_date = row['date']
    
    if trade_date != current_date:
        current_date = trade_date
        current_streak = 0
        stop_trading_today = False
        failed_days = [d for d in failed_days if (trade_date - d).days <= 7]
        recent_failures = len(failed_days)
        if recent_failures >= 3:
            risk_scale = 0.25
        elif recent_failures >= 2:
            risk_scale = 0.5
        else:
            risk_scale = 1.0
    
    if stop_trading_today:
        continue
    
    # Determine risk
    if current_streak == 0:
        risk_amount = 1000 * risk_scale
    elif current_streak == 1:
        risk_amount = 2000 * risk_scale
    elif current_streak == 2:
        risk_amount = 4000 * risk_scale
    elif current_streak == 3:
        risk_amount = 8000 * risk_scale
    else:
        continue
    
    # Track risk scaling
    if risk_scale < 1.0:
        reduced_risk_trades += 1
    else:
        full_risk_trades += 1
    
    # Calculate fees
    contract_type, contracts, multiplier, fee = determine_contracts(row, risk_amount)
    adaptive_fees += fee * contracts
    
    # Track outcomes
    if row['status'] == 'Loss':
        current_streak += 1
        if current_streak >= 4:
            stop_trading_today = True
            if trade_date not in failed_days:
                failed_days.append(trade_date)
            streak_conversions[4]['failed'] += 1
    else:
        if current_streak > 0:
            streak_conversions[current_streak]['converted'] += 1
        current_streak = 0

print("Adaptive Martingale Results (Scale Down After Multiple Failures):")
print("="*80)
print(f"\nFinal P&L: ${df['adaptive_martingale'].iloc[-1]:,.2f}")
print(f"Max Drawdown: ${df['adaptive_martingale_dd'].min():,.2f}")
print(f"Peak: ${df['adaptive_martingale_peak'].max():,.2f}")
print(f"Total Fees: ${adaptive_fees:,.2f}")

print(f"\nRisk Management:")
print(f"  Trades at full risk: {full_risk_trades}")
print(f"  Trades at reduced risk: {reduced_risk_trades} ({reduced_risk_trades/(full_risk_trades+reduced_risk_trades)*100:.1f}%)")

print(f"\nStreak Conversions:")
print(f"  1-loss streaks converted: {streak_conversions[1]['converted']}")
print(f"  2-loss streaks converted: {streak_conversions[2]['converted']}")
print(f"  3-loss streaks converted: {streak_conversions[3]['converted']}")
print(f"  4-loss failures: {streak_conversions[4]['failed']}")

print(f"\nComparison to Original Limited Martingale:")
pnl_diff = df['adaptive_martingale'].iloc[-1] - df['limited_martingale'].iloc[-1]
dd_diff = df['adaptive_martingale_dd'].min() - df['limited_martingale_dd'].min()
print(f"  P&L difference: ${pnl_diff:,.2f}")
print(f"  Drawdown difference: ${dd_diff:,.2f}")
print(f"  Drawdown reduction: {abs(dd_diff / df['limited_martingale_dd'].min()) * 100:.1f}%")
print("="*80)

Adaptive Martingale Results (Scale Down After Multiple Failures):

Final P&L: $478,192.25
Max Drawdown: $-112,633.92
Peak: $521,207.07
Total Fees: $62,810.67

Risk Management:
  Trades at full risk: 1597
  Trades at reduced risk: 395 (19.8%)

Streak Conversions:
  1-loss streaks converted: 202
  2-loss streaks converted: 124
  3-loss streaks converted: 57
  4-loss failures: 101

Comparison to Original Limited Martingale:
  P&L difference: $-44,935.67
  Drawdown difference: $18,491.86
  Drawdown reduction: 14.1%


In [63]:
# Option 2: Weekly Loss Limit
# Stop trading for the rest of the week after $45k in weekly losses
# Resume next week at full risk

def calculate_weekly_limit_martingale(df, base_risk=1000, weekly_limit=45000):
    """
    Martingale with weekly loss limit
    """
    strategy_pnl = []
    cumulative = 0
    current_streak = 0
    stop_trading_today = False
    stop_trading_week = False
    current_date = None
    current_week = None
    weekly_pnl = 0
    
    # Add date column if not exists
    if 'date' not in df.columns:
        df['date'] = pd.to_datetime(df['timestamp'], unit='ms').dt.date
    
    for idx, row in df.iterrows():
        trade_date = row['date']
        week_num = pd.to_datetime(row['timestamp'], unit='ms').isocalendar()[1]
        year_num = pd.to_datetime(row['timestamp'], unit='ms').year
        week_key = (year_num, week_num)
        
        # Reset weekly tracking at start of new week
        if week_key != current_week:
            current_week = week_key
            weekly_pnl = 0
            stop_trading_week = False
        
        # Reset daily tracking at start of new day
        if trade_date != current_date:
            current_date = trade_date
            current_streak = 0
            stop_trading_today = False
        
        # Skip if we're stopped for the week or day
        if stop_trading_week or stop_trading_today:
            strategy_pnl.append(cumulative)
            continue
        
        # Determine risk based on current streak
        if current_streak == 0:
            risk_amount = base_risk
        elif current_streak == 1:
            risk_amount = base_risk * 2
        elif current_streak == 2:
            risk_amount = base_risk * 4
        elif current_streak == 3:
            risk_amount = base_risk * 8
        else:
            strategy_pnl.append(cumulative)
            continue
        
        # Calculate P&L for this trade
        trade_pnl, contract_type, contracts = calculate_trade_pnl(row, risk_amount)
        cumulative += trade_pnl
        weekly_pnl += trade_pnl
        strategy_pnl.append(cumulative)
        
        # Check weekly limit
        if weekly_pnl <= -weekly_limit:
            stop_trading_week = True
        
        # Update streak and check if we need to stop for day
        if row['status'] == 'Loss':
            current_streak += 1
            if current_streak >= 4:
                stop_trading_today = True
        else:
            current_streak = 0
    
    return strategy_pnl

# Calculate Weekly Limit Martingale
df['weekly_limit_martingale'] = calculate_weekly_limit_martingale(df, base_risk=1000, weekly_limit=45000)

# Calculate drawdowns
df['weekly_limit_martingale_peak'] = df['weekly_limit_martingale'].cummax()
df['weekly_limit_martingale_dd'] = df['weekly_limit_martingale'] - df['weekly_limit_martingale_peak']

# Calculate statistics
weekly_fees = 0
current_streak = 0
stop_trading_today = False
stop_trading_week = False
current_date = None
current_week = None
weekly_pnl = 0
weeks_stopped = 0
trades_skipped = 0

for idx, row in df.iterrows():
    trade_date = row['date']
    week_num = pd.to_datetime(row['timestamp'], unit='ms').isocalendar()[1]
    year_num = pd.to_datetime(row['timestamp'], unit='ms').year
    week_key = (year_num, week_num)
    
    if week_key != current_week:
        if stop_trading_week:
            weeks_stopped += 1
        current_week = week_key
        weekly_pnl = 0
        stop_trading_week = False
    
    if trade_date != current_date:
        current_date = trade_date
        current_streak = 0
        stop_trading_today = False
    
    if stop_trading_week or stop_trading_today:
        trades_skipped += 1
        continue
    
    # Determine risk
    if current_streak == 0:
        risk_amount = 1000
    elif current_streak == 1:
        risk_amount = 2000
    elif current_streak == 2:
        risk_amount = 4000
    elif current_streak == 3:
        risk_amount = 8000
    else:
        continue
    
    # Calculate fees and P&L
    trade_pnl, contract_type, contracts = calculate_trade_pnl(row, risk_amount)
    contract_type, contracts, multiplier, fee = determine_contracts(row, risk_amount)
    weekly_fees += fee * contracts
    weekly_pnl += trade_pnl
    
    if weekly_pnl <= -45000:
        stop_trading_week = True
    
    if row['status'] == 'Loss':
        current_streak += 1
        if current_streak >= 4:
            stop_trading_today = True
    else:
        current_streak = 0

print("Weekly Loss Limit Martingale Results ($45k weekly limit):")
print("="*80)
print(f"\nFinal P&L: ${df['weekly_limit_martingale'].iloc[-1]:,.2f}")
print(f"Max Drawdown: ${df['weekly_limit_martingale_dd'].min():,.2f}")
print(f"Peak: ${df['weekly_limit_martingale_peak'].max():,.2f}")
print(f"Total Fees: ${weekly_fees:,.2f}")

print(f"\nWeekly Loss Limit Impact:")
print(f"  Weeks stopped: {weeks_stopped}")
print(f"  Trades skipped: {trades_skipped}")

print(f"\nComparison to Original Limited Martingale:")
pnl_diff = df['weekly_limit_martingale'].iloc[-1] - df['limited_martingale'].iloc[-1]
dd_diff = df['weekly_limit_martingale_dd'].min() - df['limited_martingale_dd'].min()
print(f"  P&L difference: ${pnl_diff:,.2f}")
print(f"  Drawdown difference: ${dd_diff:,.2f}")
if df['limited_martingale_dd'].min() != 0:
    print(f"  Drawdown reduction: {abs(dd_diff / df['limited_martingale_dd'].min()) * 100:.1f}%")
print("="*80)

Weekly Loss Limit Martingale Results ($45k weekly limit):

Final P&L: $523,127.92
Max Drawdown: $-131,125.78
Peak: $585,942.66
Total Fees: $67,475.53

Weekly Loss Limit Impact:
  Weeks stopped: 1
  Trades skipped: 249

Comparison to Original Limited Martingale:
  P&L difference: $0.00
  Drawdown difference: $0.00
  Drawdown reduction: 0.0%


In [64]:
# Option 3: Start with Lower Base Risk ($500)
# Simply use $500 as base instead of $1000
# Same martingale logic: $500 -> $1000 -> $2000 -> $4000

df['martingale_500'] = calculate_limited_martingale(df.copy())  # This uses $1000 base, let's make new one

def calculate_limited_martingale_custom_base(df, base_risk=500):
    """Martingale with custom base risk"""
    strategy_pnl = []
    cumulative = 0
    current_streak = 0
    stop_trading_today = False
    current_date = None
    
    if 'date' not in df.columns:
        df['date'] = pd.to_datetime(df['timestamp'], unit='ms').dt.date
    
    for idx, row in df.iterrows():
        trade_date = row['date']
        
        if trade_date != current_date:
            current_date = trade_date
            current_streak = 0
            stop_trading_today = False
        
        if stop_trading_today:
            strategy_pnl.append(cumulative)
            continue
        
        if current_streak == 0:
            risk_amount = base_risk
        elif current_streak == 1:
            risk_amount = base_risk * 2
        elif current_streak == 2:
            risk_amount = base_risk * 4
        elif current_streak == 3:
            risk_amount = base_risk * 8
        else:
            strategy_pnl.append(cumulative)
            continue
        
        trade_pnl, contract_type, contracts = calculate_trade_pnl(row, risk_amount)
        cumulative += trade_pnl
        strategy_pnl.append(cumulative)
        
        if row['status'] == 'Loss':
            current_streak += 1
            if current_streak >= 4:
                stop_trading_today = True
        else:
            current_streak = 0
    
    return strategy_pnl

# Test with $500 base
df['martingale_500'] = calculate_limited_martingale_custom_base(df, base_risk=500)
df['martingale_500_peak'] = df['martingale_500'].cummax()
df['martingale_500_dd'] = df['martingale_500'] - df['martingale_500_peak']

# Test with $750 base
df['martingale_750'] = calculate_limited_martingale_custom_base(df, base_risk=750)
df['martingale_750_peak'] = df['martingale_750'].cummax()
df['martingale_750_dd'] = df['martingale_750'] - df['martingale_750_peak']

print("Lower Base Risk Comparison:")
print("="*80)
print("\n$500 Base ($500 -> $1k -> $2k -> $4k, max loss $7.5k/day):")
print(f"  Final P&L: ${df['martingale_500'].iloc[-1]:,.2f}")
print(f"  Max Drawdown: ${df['martingale_500_dd'].min():,.2f}")
print(f"  Peak: ${df['martingale_500_peak'].max():,.2f}")

print("\n$750 Base ($750 -> $1.5k -> $3k -> $6k, max loss $11.25k/day):")
print(f"  Final P&L: ${df['martingale_750'].iloc[-1]:,.2f}")
print(f"  Max Drawdown: ${df['martingale_750_dd'].min():,.2f}")
print(f"  Peak: ${df['martingale_750_peak'].max():,.2f}")

print("\n$1000 Base ($1k -> $2k -> $4k -> $8k, max loss $15k/day):")
print(f"  Final P&L: ${df['limited_martingale'].iloc[-1]:,.2f}")
print(f"  Max Drawdown: ${df['limited_martingale_dd'].min():,.2f}")
print(f"  Peak: ${df['limited_martingale_peak'].max():,.2f}")

print("\nAdaptive $1000 Base (scales down after failures):")
print(f"  Final P&L: ${df['adaptive_martingale'].iloc[-1]:,.2f}")
print(f"  Max Drawdown: ${df['adaptive_martingale_dd'].min():,.2f}")
print(f"  Peak: ${df['adaptive_martingale_peak'].max():,.2f}")

print("\n" + "="*80)
print("RECOMMENDATION:")
print("  Best balance of profit vs drawdown: Adaptive $1000 Base")
print(f"  - Reduces drawdown by 14% (${abs(df['adaptive_martingale_dd'].min() - df['limited_martingale_dd'].min()):,.2f})")
print(f"  - Gives up only 9% profit (${abs(df['adaptive_martingale'].iloc[-1] - df['limited_martingale'].iloc[-1]):,.2f})")
print(f"  - Max drawdown: {abs(df['adaptive_martingale_dd'].min() / df['adaptive_martingale_peak'].max() * 100):.1f}% of peak")
print("="*80)

Lower Base Risk Comparison:

$500 Base ($500 -> $1k -> $2k -> $4k, max loss $7.5k/day):
  Final P&L: $244,888.72
  Max Drawdown: $-64,313.32
  Peak: $277,351.90

$750 Base ($750 -> $1.5k -> $3k -> $6k, max loss $11.25k/day):
  Final P&L: $380,205.74
  Max Drawdown: $-96,081.10
  Peak: $426,244.85

$1000 Base ($1k -> $2k -> $4k -> $8k, max loss $15k/day):
  Final P&L: $523,127.92
  Max Drawdown: $-131,125.78
  Peak: $585,942.66

Adaptive $1000 Base (scales down after failures):
  Final P&L: $478,192.25
  Max Drawdown: $-112,633.92
  Peak: $521,207.07

RECOMMENDATION:
  Best balance of profit vs drawdown: Adaptive $1000 Base
  - Reduces drawdown by 14% ($18,491.86)
  - Gives up only 9% profit ($44,935.67)
  - Max drawdown: 21.6% of peak


### Hybrid: Scaled Capital + Limited Martingale

Combine the best of both worlds:
- Start with $18k capital, scale risk based on balance ($500/$1000/$2000)
- Use Martingale for losses 1-3 (double each time)
- After 4th loss: stop for the day (like original martingale)
- If win on attempt 4: reset
- After any martingale sequence: apply 50% dampening for next trade (like Strategy 2)

In [66]:
# Hybrid: Scaled Capital + Limited Martingale
# Start with $18k, scale risk based on balance
# Martingale for 1-3 losses, stop on 4th loss
# After completing a martingale sequence (win or stop), apply dampening

def calculate_scaled_martingale_hybrid(df, starting_capital=18000):
    """
    Hybrid strategy: Scaled capital + Limited Martingale
    - Risk based on balance: <$50k=$500, $50k-$100k=$1000, >$100k=$2000
    - Martingale: double on losses 1-3
    - Stop trading for day after 4th loss
    - After hitting 3rd loss in martingale: apply dampening (like Strategy 2)
    """
    strategy_pnl = []
    account_balance = starting_capital
    current_streak = 0
    stop_trading_today = False
    current_date = None
    consecutive_losses = 0  # Track losses to know when we're in deep
    
    if 'date' not in df.columns:
        df['date'] = pd.to_datetime(df['timestamp'], unit='ms').dt.date
    
    for idx, row in df.iterrows():
        trade_date = row['date']
        
        # Reset daily tracking at start of new day
        if trade_date != current_date:
            current_date = trade_date
            current_streak = 0
            stop_trading_today = False
            consecutive_losses = 0
        
        # Skip if stopped for the day
        if stop_trading_today:
            strategy_pnl.append(account_balance - starting_capital)
            continue
        
        # Determine base risk from account balance
        if account_balance >= 100000:
            base_risk = 2000
        elif account_balance >= 50000:
            base_risk = 1000
        else:
            base_risk = 500
        
        # Apply martingale progression for first 3 losses
        if current_streak == 0:
            risk_amount = base_risk
        elif current_streak == 1:
            risk_amount = base_risk * 2
        elif current_streak == 2:
            risk_amount = base_risk * 4
        elif current_streak == 3:
            risk_amount = base_risk * 8
        else:
            strategy_pnl.append(account_balance - starting_capital)
            continue
        
        # After 3rd loss, apply Strategy 2 dampening
        if consecutive_losses >= 3:
            reduction_factor = 0.5 ** (consecutive_losses - 2)
            risk_amount = risk_amount * reduction_factor
        
        # Calculate P&L for this trade
        trade_pnl, contract_type, contracts = calculate_trade_pnl(row, risk_amount)
        account_balance += trade_pnl
        strategy_pnl.append(account_balance - starting_capital)
        
        # Update streak tracking
        if row['status'] == 'Loss':
            current_streak += 1
            consecutive_losses += 1
            if current_streak >= 4:
                # Failed on 4th attempt - stop for day
                stop_trading_today = True
        else:  # Win
            current_streak = 0
            consecutive_losses = 0  # Reset on any win
    
    return strategy_pnl

# Calculate Hybrid Strategy
df['scaled_martingale_hybrid'] = calculate_scaled_martingale_hybrid(df, starting_capital=18000)

# Calculate drawdowns
df['scaled_martingale_hybrid_peak'] = df['scaled_martingale_hybrid'].cummax()
df['scaled_martingale_hybrid_dd'] = df['scaled_martingale_hybrid'] - df['scaled_martingale_hybrid_peak']

# Calculate detailed statistics
hybrid_fees = 0
account_balance = 18000
current_streak = 0
stop_trading_today = False
current_date = None
dampening_active = False
risk_level_trades = {'$500': 0, '$1000': 0, '$2000': 0}
dampened_trades = 0
total_trades = 0
streak_conversions = {1: 0, 2: 0, 3: 0}
failed_days = 0

for idx, row in df.iterrows():
    trade_date = row['date']
    
    if trade_date != current_date:
        current_date = trade_date
        current_streak = 0
        stop_trading_today = False
    
    if stop_trading_today:
        continue
    
    total_trades += 1
    
    # Determine base risk
    if account_balance >= 100000:
        base_risk = 2000
        risk_level = '$2000'
    elif account_balance >= 50000:
        base_risk = 1000
        risk_level = '$1000'
    else:
        base_risk = 500
        risk_level = '$500'
    
    risk_level_trades[risk_level] += 1
    
    # Martingale progression
    if current_streak == 0:
        risk_amount = base_risk
    elif current_streak == 1:
        risk_amount = base_risk * 2
    elif current_streak == 2:
        risk_amount = base_risk * 4
    elif current_streak == 3:
        risk_amount = base_risk * 8
    else:
        continue
    
    # Apply dampening
    if dampening_active:
        risk_amount = risk_amount * 0.5
        dampened_trades += 1
    
    # Calculate fees and P&L
    trade_pnl, contract_type, contracts = calculate_trade_pnl(row, risk_amount)
    contract_type, contracts, multiplier, fee = determine_contracts(row, risk_amount)
    hybrid_fees += fee * contracts
    account_balance += trade_pnl
    
    # Update tracking
    if row['status'] == 'Loss':
        current_streak += 1
        if current_streak >= 4:
            stop_trading_today = True
            dampening_active = True
            failed_days += 1
    else:
        if current_streak > 0:
            streak_conversions[current_streak] = streak_conversions.get(current_streak, 0) + 1
            dampening_active = True
        else:
            dampening_active = False
        current_streak = 0

print("Scaled Martingale Hybrid Results:")
print("="*80)
print(f"\nFinal P&L: ${df['scaled_martingale_hybrid'].iloc[-1]:,.2f}")
print(f"Final Account Balance: ${starting_capital + df['scaled_martingale_hybrid'].iloc[-1]:,.2f}")
print(f"Max Drawdown: ${df['scaled_martingale_hybrid_dd'].min():,.2f}")
print(f"Peak P&L: ${df['scaled_martingale_hybrid_peak'].max():,.2f}")
print(f"Total Fees: ${hybrid_fees:,.2f}")

print(f"\nRisk Level Distribution:")
print(f"  $500 risk level: {risk_level_trades['$500']:,} trades ({risk_level_trades['$500']/total_trades*100:.1f}%)")
print(f"  $1000 risk level: {risk_level_trades['$1000']:,} trades ({risk_level_trades['$1000']/total_trades*100:.1f}%)")
print(f"  $2000 risk level: {risk_level_trades['$2000']:,} trades ({risk_level_trades['$2000']/total_trades*100:.1f}%)")

print(f"\nMartingale Performance:")
print(f"  1-loss streaks converted to wins: {streak_conversions.get(1, 0)}")
print(f"  2-loss streaks converted to wins: {streak_conversions.get(2, 0)}")
print(f"  3-loss streaks converted to wins: {streak_conversions.get(3, 0)}")
print(f"  4-loss failures (stopped days): {failed_days}")
print(f"  Trades with dampening applied: {dampened_trades} ({dampened_trades/total_trades*100:.1f}%)")

print(f"\nComparison to other strategies:")
print(f"  vs Original Limited Martingale ($1k base):")
print(f"    P&L: ${df['scaled_martingale_hybrid'].iloc[-1] - df['limited_martingale'].iloc[-1]:,.2f}")
print(f"    Drawdown: ${df['scaled_martingale_hybrid_dd'].min() - df['limited_martingale_dd'].min():,.2f}")
print(f"  vs Adaptive Martingale:")
print(f"    P&L: ${df['scaled_martingale_hybrid'].iloc[-1] - df['adaptive_martingale'].iloc[-1]:,.2f}")
print(f"    Drawdown: ${df['scaled_martingale_hybrid_dd'].min() - df['adaptive_martingale_dd'].min():,.2f}")

print(f"\nKey Metrics:")
print(f"  Starting capital: ${starting_capital:,.0f}")
print(f"  Max drawdown as % of peak: {abs(df['scaled_martingale_hybrid_dd'].min() / df['scaled_martingale_hybrid_peak'].max()) * 100:.1f}%")
print(f"  Would go bankrupt with $18k start: {'YES' if df['scaled_martingale_hybrid_dd'].min() < -starting_capital else 'NO'}")
print("="*80)

Scaled Martingale Hybrid Results:

Final P&L: $730,644.29
Final Account Balance: $748,644.29
Max Drawdown: $-112,157.60
Peak P&L: $796,014.79
Total Fees: $61,365.57

Risk Level Distribution:
  $500 risk level: 958 trades (48.1%)
  $1000 risk level: 112 trades (5.6%)
  $2000 risk level: 922 trades (46.3%)

Martingale Performance:
  1-loss streaks converted to wins: 202
  2-loss streaks converted to wins: 124
  3-loss streaks converted to wins: 57
  4-loss failures (stopped days): 101
  Trades with dampening applied: 1113 (55.9%)

Comparison to other strategies:
  vs Original Limited Martingale ($1k base):
    P&L: $207,516.37
    Drawdown: $18,968.18
  vs Adaptive Martingale:
    P&L: $252,452.04
    Drawdown: $476.32

Key Metrics:
  Starting capital: $18,000
  Max drawdown as % of peak: 14.1%
  Would go bankrupt with $18k start: YES


In [67]:
# Check actual minimum balance reached
min_balance_idx = df['scaled_martingale_hybrid'].idxmin()
min_balance = starting_capital + df['scaled_martingale_hybrid'].iloc[min_balance_idx]
min_date = pd.to_datetime(df.iloc[min_balance_idx]['timestamp'], unit='ms').strftime('%Y-%m-%d')

print(f"\nMinimum Balance Analysis:")
print(f"  Lowest balance reached: ${min_balance:,.2f}")
print(f"  Occurred on: {min_date} (trade #{min_balance_idx + 1})")
print(f"  Would go bankrupt: {'YES - balance below $0!' if min_balance < 0 else 'NO'}")
print(f"  Margin from bankruptcy: ${min_balance:,.2f}")


Minimum Balance Analysis:
  Lowest balance reached: $-2,156.44
  Occurred on: 2024-03-15 (trade #212)
  Would go bankrupt: YES - balance below $0!
  Margin from bankruptcy: $-2,156.44


In [68]:
# Modified Hybrid: Scaled + Martingale for only 2 losses (3 attempts total)
# Stop martingale after 3rd attempt (win or lose)
# Then switch to Strategy 2 dampening for any further losses

def calculate_scaled_martingale_lite(df, starting_capital=18000):
    """
    Conservative hybrid: Scaled capital + Martingale for only 2 losses
    - Risk based on balance: <$50k=$500, $50k-$100k=$1000, >$100k=$2000
    - Martingale: double on losses 1-2 only
    - After 3rd attempt: if still in loss streak, use Strategy 2 dampening (50%, 25%, etc.)
    """
    strategy_pnl = []
    account_balance = starting_capital
    current_streak = 0
    current_date = None
    
    if 'date' not in df.columns:
        df['date'] = pd.to_datetime(df['timestamp'], unit='ms').dt.date
    
    for idx, row in df.iterrows():
        trade_date = row['date']
        
        # Reset daily tracking at start of new day
        if trade_date != current_date:
            current_date = trade_date
            current_streak = 0
        
        # Determine base risk from account balance
        if account_balance >= 100000:
            base_risk = 2000
        elif account_balance >= 50000:
            base_risk = 1000
        else:
            base_risk = 500
        
        # Martingale for first 2 losses, then Strategy 2 dampening
        if current_streak == 0:
            risk_amount = base_risk
        elif current_streak == 1:
            risk_amount = base_risk * 2
        elif current_streak == 2:
            risk_amount = base_risk * 4
        else:
            # After 3rd attempt, switch to Strategy 2 dampening
            # streak >= 3: reduce by 50% repeatedly
            reduction_factor = 0.5 ** (current_streak - 2)
            risk_amount = base_risk * reduction_factor
        
        # Calculate P&L for this trade
        trade_pnl, contract_type, contracts = calculate_trade_pnl(row, risk_amount)
        account_balance += trade_pnl
        strategy_pnl.append(account_balance - starting_capital)
        
        # Update streak tracking
        if row['status'] == 'Loss':
            current_streak += 1
        else:
            current_streak = 0  # Reset on win
    
    return strategy_pnl

# Calculate Martingale Lite (2-loss martingale)
df['scaled_martingale_lite'] = calculate_scaled_martingale_lite(df, starting_capital=18000)

# Calculate drawdowns
df['scaled_martingale_lite_peak'] = df['scaled_martingale_lite'].cummax()
df['scaled_martingale_lite_dd'] = df['scaled_martingale_lite'] - df['scaled_martingale_lite_peak']

# Check minimum balance
min_balance_idx = df['scaled_martingale_lite'].idxmin()
min_balance = starting_capital + df['scaled_martingale_lite'].iloc[min_balance_idx]
min_date = pd.to_datetime(df.iloc[min_balance_idx]['timestamp'], unit='ms').strftime('%Y-%m-%d')

# Calculate detailed statistics
lite_fees = 0
account_balance = 18000
current_streak = 0
current_date = None
risk_level_trades = {'$500': 0, '$1000': 0, '$2000': 0}
martingale_trades = 0
dampened_trades = 0
total_trades = 0
streak_conversions = {1: 0, 2: 0}
long_streaks = 0

for idx, row in df.iterrows():
    trade_date = row['date']
    
    if trade_date != current_date:
        current_date = trade_date
        current_streak = 0
    
    total_trades += 1
    
    # Determine base risk
    if account_balance >= 100000:
        base_risk = 2000
        risk_level = '$2000'
    elif account_balance >= 50000:
        base_risk = 1000
        risk_level = '$1000'
    else:
        base_risk = 500
        risk_level = '$500'
    
    risk_level_trades[risk_level] += 1
    
    # Calculate risk
    if current_streak == 0:
        risk_amount = base_risk
    elif current_streak == 1:
        risk_amount = base_risk * 2
        martingale_trades += 1
    elif current_streak == 2:
        risk_amount = base_risk * 4
        martingale_trades += 1
    else:
        reduction_factor = 0.5 ** (current_streak - 2)
        risk_amount = base_risk * reduction_factor
        dampened_trades += 1
    
    # Calculate fees and P&L
    trade_pnl, contract_type, contracts = calculate_trade_pnl(row, risk_amount)
    contract_type, contracts, multiplier, fee = determine_contracts(row, risk_amount)
    lite_fees += fee * contracts
    account_balance += trade_pnl
    
    # Track outcomes
    if row['status'] == 'Loss':
        current_streak += 1
        if current_streak > 3:
            long_streaks += 1
    else:
        if current_streak in [1, 2]:
            streak_conversions[current_streak] = streak_conversions.get(current_streak, 0) + 1
        current_streak = 0

print("Scaled Martingale Lite Results (2-loss martingale only):")
print("="*80)
print(f"\nFinal P&L: ${df['scaled_martingale_lite'].iloc[-1]:,.2f}")
print(f"Final Account Balance: ${starting_capital + df['scaled_martingale_lite'].iloc[-1]:,.2f}")
print(f"Max Drawdown: ${df['scaled_martingale_lite_dd'].min():,.2f}")
print(f"Peak P&L: ${df['scaled_martingale_lite_peak'].max():,.2f}")
print(f"Total Fees: ${lite_fees:,.2f}")

print(f"\nMinimum Balance:")
print(f"  Lowest balance reached: ${min_balance:,.2f}")
print(f"  Occurred on: {min_date} (trade #{min_balance_idx + 1})")
print(f"  Would go bankrupt: {'YES - balance below $0!' if min_balance < 0 else 'NO - Safe!'}")
print(f"  Safety margin: ${min_balance:,.2f}")

print(f"\nRisk Level Distribution:")
print(f"  $500 risk level: {risk_level_trades['$500']:,} trades ({risk_level_trades['$500']/total_trades*100:.1f}%)")
print(f"  $1000 risk level: {risk_level_trades['$1000']:,} trades ({risk_level_trades['$1000']/total_trades*100:.1f}%)")
print(f"  $2000 risk level: {risk_level_trades['$2000']:,} trades ({risk_level_trades['$2000']/total_trades*100:.1f}%)")

print(f"\nStrategy Mix:")
print(f"  Martingale trades (2x or 4x): {martingale_trades} ({martingale_trades/total_trades*100:.1f}%)")
print(f"  Dampened trades (after 3+ losses): {dampened_trades} ({dampened_trades/total_trades*100:.1f}%)")
print(f"  Regular trades: {total_trades - martingale_trades - dampened_trades}")

print(f"\nMartingale Performance:")
print(f"  1-loss streaks converted to wins: {streak_conversions.get(1, 0)}")
print(f"  2-loss streaks converted to wins: {streak_conversions.get(2, 0)}")
print(f"  Streaks that went beyond 3 losses: {long_streaks}")

print(f"\nComparison to other strategies:")
print(f"  vs 3-loss Martingale Hybrid:")
print(f"    P&L: ${df['scaled_martingale_lite'].iloc[-1] - df['scaled_martingale_hybrid'].iloc[-1]:,.2f}")
print(f"    Drawdown: ${df['scaled_martingale_lite_dd'].min() - df['scaled_martingale_hybrid_dd'].min():,.2f}")
print(f"  vs Original Limited Martingale ($1k base):")
print(f"    P&L: ${df['scaled_martingale_lite'].iloc[-1] - df['limited_martingale'].iloc[-1]:,.2f}")
print(f"    Drawdown: ${df['scaled_martingale_lite_dd'].min() - df['limited_martingale_dd'].min():,.2f}")

print(f"\nKey Metrics:")
print(f"  Starting capital: ${starting_capital:,.0f}")
print(f"  Max drawdown as % of peak: {abs(df['scaled_martingale_lite_dd'].min() / df['scaled_martingale_lite_peak'].max()) * 100:.1f}%")
print(f"  ROI: {(df['scaled_martingale_lite'].iloc[-1] / starting_capital) * 100:.1f}%")
print("="*80)

Scaled Martingale Lite Results (2-loss martingale only):

Final P&L: $802,037.83
Final Account Balance: $820,037.83
Max Drawdown: $-67,779.25
Peak P&L: $820,826.77
Total Fees: $69,769.57

Minimum Balance:
  Lowest balance reached: $3,901.35
  Occurred on: 2024-04-02 (trade #253)
  Would go bankrupt: NO - Safe!
  Safety margin: $3,901.35

Risk Level Distribution:
  $500 risk level: 784 trades (35.0%)
  $1000 risk level: 219 trades (9.8%)
  $2000 risk level: 1,238 trades (55.2%)

Strategy Mix:
  Martingale trades (2x or 4x): 892 (39.8%)
  Dampened trades (after 3+ losses): 311 (13.9%)
  Regular trades: 1038

Martingale Performance:
  1-loss streaks converted to wins: 211
  2-loss streaks converted to wins: 130
  Streaks that went beyond 3 losses: 196

Comparison to other strategies:
  vs 3-loss Martingale Hybrid:
    P&L: $71,393.55
    Drawdown: $44,378.35
  vs Original Limited Martingale ($1k base):
    P&L: $278,909.92
    Drawdown: $63,346.53

Key Metrics:
  Starting capital: $18,000

In [55]:
# Create detailed results dataframe for exploration
# Track trade-by-trade details for all strategies

results_data = []
starting_capital = 18000

# Initialize tracking variables for each strategy
baseline_1k_balance = 0
strategy2_1k_balance = 0
strategy2_1k_streak = 0

baseline_2k_balance = 0
aggressive_balance = 0
aggressive_streak = 0

scaled_balance = starting_capital
scaled_streak = 0

for idx, row in df.iterrows():
    trade_data = {
        'timestamp': pd.to_datetime(row['timestamp'], unit='ms').strftime('%Y-%m-%d %H:%M:%S'),
        'order_type': row['orderType'],
        'status': row['status'],
        'low': row['low'],
        'high': row['high'],
        'price_range': row['high'] - row['low']
    }
    
    # Baseline $1000
    baseline_1k_risk = 1000
    baseline_1k_pnl, baseline_1k_contract_type, baseline_1k_contracts = calculate_trade_pnl(row, baseline_1k_risk)
    baseline_1k_balance += baseline_1k_pnl
    trade_data['baseline_1k_risk'] = baseline_1k_risk
    trade_data['baseline_1k_contracts'] = f"{baseline_1k_contracts} {baseline_1k_contract_type}"
    trade_data['baseline_1k_pnl'] = baseline_1k_pnl
    trade_data['baseline_1k_cumulative'] = baseline_1k_balance
    
    # Strategy 2 $1000 with dampening
    if strategy2_1k_streak >= 3:
        reduction_factor = 0.5 ** (strategy2_1k_streak - 2)
    else:
        reduction_factor = 1.0
    strategy2_1k_risk = 1000 * reduction_factor
    strategy2_1k_pnl, strategy2_1k_contract_type, strategy2_1k_contracts = calculate_trade_pnl(row, strategy2_1k_risk)
    strategy2_1k_balance += strategy2_1k_pnl
    trade_data['strategy2_1k_risk'] = strategy2_1k_risk
    trade_data['strategy2_1k_contracts'] = f"{strategy2_1k_contracts} {strategy2_1k_contract_type}"
    trade_data['strategy2_1k_pnl'] = strategy2_1k_pnl
    trade_data['strategy2_1k_cumulative'] = strategy2_1k_balance
    
    # Update streak
    if row['status'] == 'Loss':
        strategy2_1k_streak += 1
    else:
        strategy2_1k_streak = 0
    
    # Baseline $2000
    baseline_2k_risk = 2000
    baseline_2k_pnl, baseline_2k_contract_type, baseline_2k_contracts = calculate_trade_pnl(row, baseline_2k_risk)
    baseline_2k_balance += baseline_2k_pnl
    trade_data['baseline_2k_risk'] = baseline_2k_risk
    trade_data['baseline_2k_contracts'] = f"{baseline_2k_contracts} {baseline_2k_contract_type}"
    trade_data['baseline_2k_pnl'] = baseline_2k_pnl
    trade_data['baseline_2k_cumulative'] = baseline_2k_balance
    
    # Aggressive Strategy 2 with $2000 base
    if aggressive_streak >= 2:
        reduction_factor = 0.5 ** (aggressive_streak - 2 + 1)
    else:
        reduction_factor = 1.0
    aggressive_risk = 2000 * reduction_factor
    aggressive_pnl, aggressive_contract_type, aggressive_contracts = calculate_trade_pnl(row, aggressive_risk)
    aggressive_balance += aggressive_pnl
    trade_data['aggressive_risk'] = aggressive_risk
    trade_data['aggressive_contracts'] = f"{aggressive_contracts} {aggressive_contract_type}"
    trade_data['aggressive_pnl'] = aggressive_pnl
    trade_data['aggressive_cumulative'] = aggressive_balance
    
    # Update aggressive streak
    if row['status'] == 'Loss':
        aggressive_streak += 1
    else:
        aggressive_streak = 0
    
    # Scaled Strategy 2
    if scaled_balance >= 100000:
        scaled_base_risk = 2000
        risk_level = '$2000'
    elif scaled_balance >= 50000:
        scaled_base_risk = 1000
        risk_level = '$1000'
    else:
        scaled_base_risk = 500
        risk_level = '$500'
    
    if scaled_streak >= 3:
        reduction_factor = 0.5 ** (scaled_streak - 2)
    else:
        reduction_factor = 1.0
    
    scaled_risk = scaled_base_risk * reduction_factor
    scaled_pnl, scaled_contract_type, scaled_contracts = calculate_trade_pnl(row, scaled_risk)
    scaled_balance += scaled_pnl
    trade_data['scaled_risk_level'] = risk_level
    trade_data['scaled_risk'] = scaled_risk
    trade_data['scaled_contracts'] = f"{scaled_contracts} {scaled_contract_type}"
    trade_data['scaled_pnl'] = scaled_pnl
    trade_data['scaled_balance'] = scaled_balance
    trade_data['scaled_cumulative_pnl'] = scaled_balance - starting_capital
    
    # Update scaled streak
    if row['status'] == 'Loss':
        scaled_streak += 1
    else:
        scaled_streak = 0
    
    results_data.append(trade_data)

# Create DataFrame
results_df = pd.DataFrame(results_data)

print("Trade-by-Trade Results DataFrame Created!")
print("="*80)
print(f"Total trades: {len(results_df):,}")
print(f"\nColumns available:")
for col in results_df.columns:
    print(f"  - {col}")
print("\nFirst 10 trades:")
print(results_df.head(10).to_string(index=False))
print("="*80)
print("\nYou can now explore with:")
print("  results_df.head(n) - View first n trades")
print("  results_df.tail(n) - View last n trades")
print("  results_df[results_df['status']=='Loss'] - Filter losing trades")
print("  results_df[results_df['scaled_risk_level']=='$2000'] - Filter by risk level")
print("  results_df.describe() - Statistical summary")
print("="*80)

Trade-by-Trade Results DataFrame Created!
Total trades: 2,241

Columns available:
  - timestamp
  - order_type
  - status
  - low
  - high
  - price_range
  - baseline_1k_risk
  - baseline_1k_contracts
  - baseline_1k_pnl
  - baseline_1k_cumulative
  - strategy2_1k_risk
  - strategy2_1k_contracts
  - strategy2_1k_pnl
  - strategy2_1k_cumulative
  - baseline_2k_risk
  - baseline_2k_contracts
  - baseline_2k_pnl
  - baseline_2k_cumulative
  - aggressive_risk
  - aggressive_contracts
  - aggressive_pnl
  - aggressive_cumulative
  - scaled_risk_level
  - scaled_risk
  - scaled_contracts
  - scaled_pnl
  - scaled_balance
  - scaled_cumulative_pnl

First 10 trades:
          timestamp order_type status       low      high  price_range  baseline_1k_risk baseline_1k_contracts  baseline_1k_pnl  baseline_1k_cumulative  strategy2_1k_risk strategy2_1k_contracts  strategy2_1k_pnl  strategy2_1k_cumulative  baseline_2k_risk baseline_2k_contracts  baseline_2k_pnl  baseline_2k_cumulative  aggressive_ri

In [58]:
# Export results_df to CSV in downloads folder
results_df.to_csv('/Users/gf26229/Downloads/trading_results_analysis.csv', index=False)
print(f"Results exported successfully to: /Users/gf26229/Downloads/trading_results_analysis.csv")
print(f"Total rows exported: {len(results_df):,}")

Results exported successfully to: /Users/gf26229/Downloads/trading_results_analysis.csv
Total rows exported: 2,241


In [56]:
results_df

,timestamp,order_type,status,low,high,price_range,baseline_1k_risk,baseline_1k_contracts,baseline_1k_pnl,baseline_1k_cumulative,...,aggressive_risk,aggressive_contracts,aggressive_pnl,aggressive_cumulative,scaled_risk_level,scaled_risk,scaled_contracts,scaled_pnl,scaled_balance,scaled_cumulative_pnl
0,2024-01-02 14:31:00,Long,Loss,4785.0186,4788.5000,3.4814,1000,6 ES,-1061.7000,-1061.7000,...,2000.0,11 ES,-1946.450,-1946.4500,$500,500.0,28 MES,-513.996,17486.004,-513.996
1,2024-01-02 17:24:00,Short,Loss,4791.0000,4794.1836,3.1836,1000,6 ES,-972.3600,-2034.0600,...,2000.0,12 ES,-1944.720,-3891.1700,$500,500.0,3 ES,-486.180,16999.824,-1000.176
2,2024-01-02 17:38:00,Long,Loss,4792.6523,4795.5000,2.8477,1000,7 ES,-1016.8550,-3050.9150,...,1000.0,7 ES,-1016.855,-4908.0250,$500,500.0,4 ES,-581.060,16418.764,-1581.236
3,2024-01-02 17:51:00,Short,Profit,4789.0000,4792.3090,3.3090,1000,6 ES,1968.1200,-1082.7950,...,500.0,3 ES,984.060,-3923.9650,$500,250.0,15 MES,482.100,16900.864,-1099.136
4,2024-01-02 18:56:00,Short,Profit,4782.2500,4785.4146,3.1646,1000,6 ES,1881.4800,798.6850,...,2000.0,13 ES,4076.540,152.5750,$500,500.0,3 ES,940.740,17841.604,-158.396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2236,2025-12-04 14:35:00,Short,Profit,6867.5000,6873.8990,6.3990,1000,3 ES,1911.0600,290116.9180,...,2000.0,6 ES,3822.120,445872.0635,$2000,2000.0,6 ES,3822.120,334065.021,316065.021
2237,2025-12-04 17:15:00,Long,Loss,6857.3496,6862.7500,5.4004,1000,4 ES,-1091.6000,289025.3180,...,2000.0,7 ES,-1910.300,443961.7635,$2000,2000.0,7 ES,-1910.300,332154.721,314154.721
2238,2025-12-04 19:38:00,Long,Loss,6851.1475,6858.0000,6.8525,1000,29 MES,-1021.1625,288004.1555,...,2000.0,6 ES,-2073.030,441888.7335,$2000,2000.0,6 ES,-2073.030,330081.691,312081.691
2239,2025-12-04 20:07:00,Short,Loss,6848.2500,6854.2417,5.9917,1000,3 ES,-907.3950,287096.7605,...,1000.0,3 ES,-907.395,440981.3385,$2000,2000.0,7 ES,-2117.255,327964.436,309964.436


In [47]:
# Debug: Check a few trades to understand the win/loss ratio issue
print("Debugging first few trades:")
print("="*100)

for i in range(10):
    row = df.iloc[i]
    price_range = row['high'] - row['low']
    
    if row['orderType'] == 'Long':
        entry = row['high']
        stop_loss = row['low']
        take_profit = entry + 2 * price_range
        
        print(f"\nTrade {i+1} - Long {row['status']}")
        print(f"  Range: {price_range:.4f} points")
        print(f"  Entry: {entry:.4f}")
        print(f"  Stop: {stop_loss:.4f}")
        print(f"  Target: {take_profit:.4f}")
        print(f"  Risk: {price_range:.4f} points")
        print(f"  Reward: {2*price_range:.4f} points (2x risk)")
        
        if row['status'] == 'Profit':
            points = take_profit - entry
        else:
            points = stop_loss - entry
        print(f"  Actual points: {points:.4f}")
        print(f"  P&L: ${row['baseline_pnl']:.2f}")
    else:
        entry = row['low']
        stop_loss = row['high']
        take_profit = entry - 2 * price_range
        
        print(f"\nTrade {i+1} - Short {row['status']}")
        print(f"  Range: {price_range:.4f} points")
        print(f"  Entry: {entry:.4f}")
        print(f"  Stop: {stop_loss:.4f}")
        print(f"  Target: {take_profit:.4f}")
        print(f"  Risk: {price_range:.4f} points")
        print(f"  Reward: {2*price_range:.4f} points (2x risk)")
        
        if row['status'] == 'Profit':
            points = entry - take_profit
        else:
            points = entry - stop_loss
        print(f"  Actual points: {points:.4f}")
        print(f"  P&L: ${row['baseline_pnl']:.2f}")

Debugging first few trades:

Trade 1 - Long Loss
  Range: 3.4814 points
  Entry: 4788.5000
  Stop: 4785.0186
  Target: 4795.4628
  Risk: 3.4814 points
  Reward: 6.9628 points (2x risk)
  Actual points: -3.4814
  P&L: $-1061.70

Trade 2 - Short Loss
  Range: 3.1836 points
  Entry: 4791.0000
  Stop: 4794.1836
  Target: 4784.6328
  Risk: 3.1836 points
  Reward: 6.3672 points (2x risk)
  Actual points: -3.1836
  P&L: $-972.36

Trade 3 - Long Loss
  Range: 2.8477 points
  Entry: 4795.5000
  Stop: 4792.6523
  Target: 4801.1954
  Risk: 2.8477 points
  Reward: 5.6954 points (2x risk)
  Actual points: -2.8477
  P&L: $-1016.86

Trade 4 - Short Profit
  Range: 3.3090 points
  Entry: 4789.0000
  Stop: 4792.3090
  Target: 4782.3820
  Risk: 3.3090 points
  Reward: 6.6180 points (2x risk)
  Actual points: 6.6180
  P&L: $1968.12

Trade 5 - Short Profit
  Range: 3.1646 points
  Entry: 4782.2500
  Stop: 4785.4146
  Target: 4775.9208
  Risk: 3.1646 points
  Reward: 6.3292 points (2x risk)
  Actual points